# Flatiron Health mCRC: Data Wrangling Test Set

**OBJECTIVE: Create a dataframe of relevant variables using test cohort patients which will be used to validate machine learning survival models.**

**BACKGROUND: The 11 CSV Flatiron files will be cleaned in the exact same fashion for the test set patients as for the training set patients. For more information on the cleaning process refer to Notebook: Data Wrangling Training Set.**

**OUTLINE:**
1. **File cleaning for patients in training set**
2. **Merge files to create master test dataframe** 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

In [3]:
#Import test IDs saved from Data Wrangling Training Set file. 
test_IDs = pd.read_csv('test_IDs.csv')

In [4]:
# Array of PatientIDs in training set.
test_IDs = test_IDs['PatientID'].to_numpy()

In [5]:
len(test_IDs)

6863

## Part 1: Data wrangling

**Relevant CSV files will be imported and processed. A file is considered processed when each row corresponds to a unique patient from the training set and each column is a relevant variable for mortality prognositication. The eligibility window for collecting variables is typically defined as -90 days and +30 days from index date. The index date is time of metastatic diagnosis. Plus 30 was selected as the upper bound of the eligibility window given that median time to start of first line treatment is about 30 days from metastatic diagnosis.** 

**The following 11 CSV files from Flatiron will be cleaned:**
1. **Demographics**
2. **Enhanced_MetastaticCRC**
3. **Enhanced_Mortality_V2**
4. **MedicationAdministration**
5. **Enhanced_MetCRCBiomarkers**
6. **Insurance**
7. **ECOG**
8. **Vitals**
9. **Labs**
10. **Diagnosis**
11. **SocialDeeterminantsOfHealth**

### 1. Demographics

In [6]:
demographics = pd.read_csv('Demographics.csv')

In [7]:
demographics = demographics[demographics['PatientID'].isin(test_IDs)]

In [8]:
row_ID(demographics)

(6863, 6863)

#### Race and Ethnicity

In [9]:
# If race value is 'Hispanic or Latino', code as unknown, otherwise value unchanged.
demographics['race'] = (
    np.where(demographics['Race'] == 'Hispanic or Latino', 'unknown', demographics['Race'])
)

In [10]:
# Missing race value will be recoded as Unknown
demographics['race'] = demographics['race'].fillna('unknown')

In [11]:
demographics['race'].value_counts().sum()

6863

In [12]:
# If race value is equal to 'Hispanic or Latino', code ethnicity as 'Hispanic or Latino', otherwise unchanged. 
demographics['ethnicity'] = (
    np.where(demographics['Race'] == 'Hispanic or Latino', 'hispanic_latino', demographics['Ethnicity'])
)

In [13]:
demographics['ethnicity'] = demographics['ethnicity'].fillna('unknown')

In [14]:
demographics['ethnicity'] = demographics['ethnicity'].replace({'Hispanic or Latino': 'hispanic_latino'})

In [15]:
demographics = demographics.drop(columns = ['Race', 'Ethnicity'])

#### BirthYear

In [16]:
enhanced_met = pd.read_csv('Enhanced_MetastaticCRC.csv')

In [17]:
demographics = pd.merge(demographics, enhanced_met[['PatientID', 'MetDiagnosisDate']], on = 'PatientID')

In [18]:
demographics.loc[:, 'MetDiagnosisDate'] = pd.to_datetime(demographics['MetDiagnosisDate'])

In [19]:
demographics.loc[:, 'age'] = demographics['MetDiagnosisDate'].dt.year - demographics['BirthYear']

In [20]:
demographics = demographics.drop(columns = ['BirthYear', 'MetDiagnosisDate'])

#### PracticeType

In [21]:
practice = pd.read_csv('Practice.csv')

In [22]:
practice = practice[practice['PatientID'].isin(test_IDs)]

In [23]:
row_ID(practice)

(6991, 6863)

In [24]:
practice_unique_count = (
    practice.groupby('PatientID')['PracticeType'].agg('nunique')
    .to_frame()
    .reset_index()
    .rename(columns = {'PracticeType': 'n_type'})
)

In [25]:
practice_n = pd.merge(practice, practice_unique_count, on = 'PatientID')

In [26]:
practice_n['p_type'] = (
    np.where(practice_n['n_type'] == 1, practice_n['PracticeType'], 'BOTH')
)

In [27]:
practice_n = (
    practice_n.drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'p_type'])
)

In [28]:
demographics = pd.merge(demographics, practice_n, on = 'PatientID')

#### Gender

In [29]:
# Impute M as unknown given most common gender. 
demographics['Gender'] = demographics['Gender'].fillna('M')

In [30]:
demographics = demographics.rename(columns = {'Gender': 'gender'})

#### State

In [31]:
# Group states into Census-Bureau regions  
state_dict = { 
    'ME': 'northeast', 
    'NH': 'northeast',
    'VT': 'northeast', 
    'MA': 'northeast',
    'CT': 'northeast',
    'RI': 'northeast',  
    'NY': 'northeast', 
    'NJ': 'northeast', 
    'PA': 'northeast', 
    'IL': 'midwest', 
    'IN': 'midwest', 
    'MI': 'midwest', 
    'OH': 'midwest', 
    'WI': 'midwest',
    'IA': 'midwest',
    'KS': 'midwest',
    'MN': 'midwest',
    'MO': 'midwest', 
    'NE': 'midwest',
    'ND': 'midwest',
    'SD': 'midwest',
    'DE': 'south',
    'FL': 'south',
    'GA': 'south',
    'MD': 'south',
    'NC': 'south', 
    'SC': 'south',
    'VA': 'south',
    'DC': 'south',
    'WV': 'south',
    'AL': 'south',
    'KY': 'south',
    'MS': 'south',
    'TN': 'south',
    'AR': 'south',
    'LA': 'south',
    'OK': 'south',
    'TX': 'south',
    'AZ': 'west',
    'CO': 'west',
    'ID': 'west',
    'MT': 'west',
    'NV': 'west',
    'NM': 'west',
    'UT': 'west',
    'WY': 'west',
    'AK': 'west',
    'CA': 'west',
    'HI': 'west',
    'OR': 'west',
    'WA': 'west',
    'PR': 'unknown'
}

demographics['region'] = demographics['State'].map(state_dict)

In [32]:
demographics['region'] = demographics['region'].fillna('unknown')

In [33]:
demographics = demographics.drop(columns = ['State'])

In [34]:
# Final training demographics table.
demographics.sample(5)

,PatientID,gender,race,ethnicity,age,p_type,region
3360,F24EC6B73C794,M,White,Not Hispanic or Latino,48,ACADEMIC,unknown
1221,F5EC96C220FD3,M,Other Race,unknown,44,COMMUNITY,midwest
1500,FF8747E1DCA9B,F,White,hispanic_latino,49,COMMUNITY,unknown
4627,F321D1A0A7F79,F,Black or African American,Not Hispanic or Latino,53,COMMUNITY,south
6524,F2D95CFBCC7FC,M,White,Not Hispanic or Latino,77,COMMUNITY,south


In [35]:
%whos DataFrame

Variable                Type         Data/Info
----------------------------------------------
demographics            DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
enhanced_met            DataFrame               PatientID Diag<...>n[34315 rows x 5 columns]
practice                DataFrame               PatientID     <...>\n[6991 rows x 4 columns]
practice_n              DataFrame              PatientID     p<...>\n[6863 rows x 2 columns]
practice_unique_count   DataFrame              PatientID  n_ty<...>\n[6863 rows x 2 columns]


In [36]:
# Keep demographics and enhanced_met
del practice
del practice_n
del practice_unique_count

### 2. Enhanced_MetastaticCRC

In [37]:
enhanced_met = enhanced_met[enhanced_met['PatientID'].isin(test_IDs)]

In [38]:
row_ID(enhanced_met)

(6863, 6863)

#### GroupStage 

In [39]:
# Dictionary for regrouping stages
stage_dict = { 
    '0': '0',
    'I': 'I',
    'II': 'II',
    'IIA': 'II',
    'IIB': 'II',
    'IIC': 'II',
    'III': 'III',
    'IIIA': 'III',
    'IIIB': 'III',
    'IIIC': 'III',
    'IV': 'IV',
    'IVA': 'IV',
    'IVB': 'IV'
}

enhanced_met['stage'] = enhanced_met['GroupStage'].map(stage_dict)

In [40]:
enhanced_met['stage'] = enhanced_met['stage'].fillna('unknown')

In [41]:
enhanced_met = enhanced_met.drop(columns = ['GroupStage'])

#### CRCSite

**Refer to Diagnosis section for further cleaning of variable.**

#### MetDiagnosisDate

In [42]:
enhanced_met = enhanced_met.rename(columns = {'MetDiagnosisDate': 'met_date'})

In [43]:
enhanced_met.loc[:, 'met_date'] = pd.to_datetime(enhanced_met['met_date'])

In [44]:
enhanced_met.loc[:, 'met_year'] = enhanced_met['met_date'].dt.year

#### DiagnosisDate

In [45]:
enhanced_met = enhanced_met.rename(columns = {'DiagnosisDate': 'diagnosis_date'})

In [46]:
# Missing diagnosis_date will be replaced with met_date; other dates will be left untouched. 
enhanced_met['diagnosis_date'] = (
    np.where(enhanced_met['diagnosis_date'].isna(), enhanced_met['met_date'], enhanced_met['diagnosis_date'])
)

In [47]:
enhanced_met['diagnosis_date'] = pd.to_datetime(enhanced_met['diagnosis_date'])

#### Time from diagnosis date to metastatic date

In [48]:
enhanced_met.loc[:, 'delta_met_diagnosis'] = (enhanced_met['met_date'] - enhanced_met['diagnosis_date']).dt.days

In [49]:
enhanced_met.sample(5)

,PatientID,diagnosis_date,met_date,CrcSite,stage,met_year,delta_met_diagnosis
11812,FF7ADAF4603D3,2021-07-27,2021-07-27,Rectum,IV,2021,0
3296,FBCFBF8184FFA,2021-09-16,2021-09-16,Colon,IV,2021,0
7134,FCAABB0418596,2014-05-07,2017-01-06,Rectum,unknown,2017,975
4866,FFEA77652F807,2016-11-30,2016-11-30,Colon,IV,2016,0
17244,F6A30ED8E5604,2012-06-01,2014-07-14,Rectum,I,2014,773


In [50]:
%whos DataFrame

Variable       Type         Data/Info
-------------------------------------
demographics   DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
enhanced_met   DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]


### 3. Enhanced_Mortality_V2

In [51]:
mortality = pd.read_csv('Enhanced_Mortality_V2.csv')

In [52]:
mortality = mortality[mortality['PatientID'].isin(test_IDs)]

In [53]:
row_ID(mortality)

(4236, 4236)

In [54]:
mortality = mortality.rename(columns = {'DateOfDeath': 'death_date'})

In [55]:
# For patients with year granularity, impute middle of the year (ie., July 1)
mortality['death_date'] = (
    np.where(mortality['death_date'].str.len() == 4, mortality['death_date'] + '-07-01', mortality['death_date'])
)

In [56]:
# For patients with month granularity, impute 15th of the month.
mortality['death_date'] = (
    np.where(mortality['death_date'].str.len() == 7, mortality['death_date'] + '-15', mortality['death_date'])
)

In [57]:
mortality['death_date'] = pd.to_datetime(mortality['death_date'])

#### Censoring

**For patients for whom a date of death is not known, the censor date can be defined either as the data cutoff date or as the last confirmed activity date. The last confirmed activity date is broadly defined as the last date at which there is evidence in the EHR that a patient is alive. Evidence of a record in at least one of the items listed below qualifies as patient-level confirmed activity:**
* **Visit: VisitDate**
* **Enhanced_MetCRC_Orals: StartDate or EndDate**
* **Enhanced_MetCRCBiomarkers: SpecimenCollectedDate**

In [58]:
visit = pd.read_csv('Visit.csv')
telemedicine = pd.read_csv('Telemedicine.csv')
orals = pd.read_csv('Enhanced_MetCRC_Orals.csv')
biomarkers = pd.read_csv('Enhanced_MetCRCBiomarkers.csv')

##### Visit and Telemedicine

In [59]:
visit_tele = (
    visit
    .drop(columns = ['VisitType', 'IsVitalsVisit', 'IsTreatmentVisit', 'IsLabVisit'])
    .append(telemedicine)
) 

In [60]:
visit_tele.loc[:,'VisitDate'] = pd.to_datetime(visit_tele['VisitDate'])

In [61]:
# Select max VisitDate from combined Visit and Telemedicine table.
visit_tele_max = (
    visit_tele
    [visit_tele['PatientID'].isin(test_IDs)]
    .groupby('PatientID')['VisitDate'].max()
    .to_frame(name = 'visit_max')
    .reset_index()
)

##### Orals

In [62]:
orals = orals[orals['PatientID'].isin(test_IDs)]

In [63]:
orals.loc[:, 'StartDate'] = pd.to_datetime(orals['StartDate'])

In [64]:
orals.loc[:, 'EndDate'] = pd.to_datetime(orals['EndDate'])

In [65]:
orals_max = (
    orals
    .assign(max_date = orals[['StartDate', 'EndDate']].max(axis = 1))
    .groupby('PatientID')['max_date'].max()
    .to_frame(name = 'orals_max')
    .reset_index()
)

##### Biomarkers

In [66]:
biomarkers = biomarkers[biomarkers['PatientID'].isin(test_IDs)]

In [67]:
biomarkers.loc[:, 'SpecimenCollectedDate'] = pd.to_datetime(biomarkers['SpecimenCollectedDate'])

In [68]:
biomarkers_max = (
    biomarkers
    .groupby('PatientID')['SpecimenCollectedDate'].max()
    .to_frame(name = 'biomarkers_max')
    .reset_index()
)

##### Max date merge

In [69]:
last_activity = pd.merge(visit_tele_max, orals_max, on = 'PatientID', how = 'outer')

In [70]:
last_activity = pd.merge(last_activity, biomarkers_max, on = 'PatientID', how = 'outer')

In [71]:
row_ID(last_activity)

(6863, 6863)

In [72]:
# Find max of each row. 
last_activity = (
    last_activity
    .assign(last_activity = last_activity[['visit_max', 'orals_max', 'biomarkers_max']].max(axis = 1))
    .filter(items = ['PatientID', 'last_activity'])
)

In [73]:
# Append missing training IDs.
mortality = (
    mortality
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(mortality['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False
    )
)

In [74]:
row_ID(mortality)

(6863, 6863)

In [75]:
mortality = pd.merge(mortality, enhanced_met[['PatientID', 'met_date']], on = 'PatientID')

In [76]:
mortality = pd.merge(mortality, last_activity, on = 'PatientID')

In [77]:
mortality.loc[:, 'death_status'] = np.where(mortality['death_date'].isna(), 0, 1)

In [78]:
# timerisk_activity is time from advanced disease diagnosis to death or last activity if no death date.
mortality.loc[:, 'timerisk_activity'] = (
    np.where(mortality['death_date'].isna(),
             (mortality['last_activity'] - mortality['met_date']).dt.days,
             (mortality['death_date'] - mortality['met_date']).dt.days)
)

In [79]:
# If timerisk_activity is less than 0, set to 0 otherwise remains unchanged. 
mortality['timerisk_activity'] = np.where(mortality['timerisk_activity'] < 0, 0, mortality['timerisk_activity'])

In [80]:
mortality.sample(5)

,PatientID,death_date,met_date,last_activity,death_status,timerisk_activity
2172,F6DDA87DEC300,2014-08-15,2014-07-21,2014-07-21,1,25.0
2799,FF8ABF9B4B79D,2020-12-15,2019-08-05,2020-12-10,1,498.0
1888,FE3E803840C6E,2015-09-15,2014-09-22,2015-07-30,1,358.0
3211,FD3FCDEB1B62F,2019-05-15,2016-07-03,2019-01-28,1,1046.0
4924,F44D8282DA6D7,NaT,2015-09-25,2015-11-06,0,42.0


In [81]:
mortality = pd.merge(mortality, enhanced_met[['PatientID', 'diagnosis_date']], on = 'PatientID', how = 'outer')

In [82]:
# timerisk_activity_first is time from first diagnosis (metastatic or not) to death or last activity if no death date.
mortality.loc[:, 'timerisk_activity_first'] = (
    np.where(mortality['death_date'].isna(),
             (mortality['last_activity'] - mortality['diagnosis_date']).dt.days,
             (mortality['death_date'] - mortality['diagnosis_date']).dt.days)
)

In [83]:
# If timerisk_activity is less than 0, set to 0 otherwise remains unchanged. 
mortality['timerisk_activity_first'] = np.where(
    mortality['timerisk_activity_first'] < 0, 0, mortality['timerisk_activity_first'])

In [84]:
mortality.to_csv('mortality_cleaned_te.csv', index = False, header = True)

In [85]:
mortality = mortality.filter(items = ['PatientID', 'death_status', 'timerisk_activity'])

In [86]:
mortality.sample(5)

,PatientID,death_status,timerisk_activity
424,F1A6D3E385AFC,1,357.0
6252,F41AF2679737A,0,3059.0
5257,F6AFA3BB41C6E,0,413.0
938,F1FEFF7241A01,1,308.0
2302,FEA02AEBB7435,1,554.0


In [87]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
biomarkers       DataFrame                PatientID Bio<...>[24757 rows x 15 columns]
biomarkers_max   DataFrame              PatientID bioma<...>\n[5930 rows x 2 columns]
demographics     DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
enhanced_met     DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
last_activity    DataFrame              PatientID last_<...>\n[6863 rows x 2 columns]
mortality        DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]
orals            DataFrame               PatientID     <...>\n[4484 rows x 5 columns]
orals_max        DataFrame              PatientID  oral<...>\n[2931 rows x 2 columns]
telemedicine     DataFrame               PatientID     <...>n[23327 rows x 3 columns]
visit            DataFrame                 PatientID   <...>2010098 rows x 7 columns]
visit_tele       DataFrame               PatientID     <...>

In [88]:
# Keep demographics, enhanced_met, and mortality
del biomarkers
del biomarkers_max
del last_activity
del orals
del orals_max
del telemedicine
del visit
del visit_tele
del visit_tele_max

### 4. MedicationAdministration

In [89]:
med_admin = pd.read_csv('MedicationAdministration.csv')

In [90]:
med_admin = med_admin[med_admin['PatientID'].isin(test_IDs)]

In [91]:
row_ID(med_admin)

(881806, 5676)

In [92]:
med_admin.shape

(881806, 11)

**An indicator variable will be created for key medications (ie., steroids, opioids, other pain meds, antibiotics, anticoagulation, diabetic medicaitons, etc.) around time of metastatic diagnosis. The elgibility window is -90 days from metastatic diagnosis to first line of therapy or +30, whichever comes first. First line of therapy is included as an upper bound because steroids are frequently administered as part of treatment for chemotherapy induced-nausea, so steroids might inadvertently capture chemotherapy treatment if upper bound is set after first line of therapy.** 

In [93]:
line_therapy = pd.read_csv('LineOfTherapy.csv')

In [94]:
line_therapy = line_therapy[line_therapy['PatientID'].isin(med_admin['PatientID'])]

In [95]:
line_therapy_1 = (
    line_therapy 
    .query('LineNumber == 1 and IsMaintenanceTherapy == False')
)

In [96]:
# If patients have 2 first line therapies, select earliest
line_therapy_1 = line_therapy_1.drop_duplicates(subset = ['PatientID'], keep = 'first')

In [97]:
med_admin = pd.merge(med_admin, line_therapy_1[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [98]:
med_admin = pd.merge(med_admin, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [99]:
med_admin.loc[:, 'AdministeredDate'] = pd.to_datetime(med_admin['AdministeredDate'])

In [100]:
med_admin.loc[:, 'StartDate'] = pd.to_datetime(med_admin['StartDate'])

In [101]:
med_admin['AdministeredDate'].isna().sum()

0

In [102]:
# New variable upper_bound which defines upper bound
# If no StartDate (ie., no treatment received), then upper bound +30 from metastatic diagnosis 
# If StartDate is greater than 30 days from metastatic diagnosis, then upper bound +30 from metastatic diagnosis
# If StartDate is less than or equal 30 from metastatic diagnosis, then upper bound is one day before StartDate
conditions = [
    (med_admin['StartDate'].isna()) | ((med_admin['StartDate'] - med_admin['met_date']).dt.days > 30),
    ((med_admin['StartDate'] - med_admin['met_date']).dt.days <= 30)]    

choices = [30, (med_admin['StartDate'] - med_admin['met_date']).dt.days - 1]
    
med_admin.loc[:, 'upper_bound'] = np.select(conditions, choices)

In [103]:
med_admin.loc[:, 'upper_bound_date'] = (
    np.where(med_admin['upper_bound'] != 30, 
             med_admin['StartDate'] - pd.DateOffset(days = 1), 
             med_admin['met_date'] + pd.DateOffset(days = 30))
)

In [104]:
# Select window of -90 days and from metastatic diagnosis and remove clinical study drug. 
med_admin_win = (
    med_admin
    [((med_admin['AdministeredDate'] - med_admin['met_date']).dt.days >= -90) &
    (med_admin['AdministeredDate'] <= med_admin['upper_bound_date']) &
    (med_admin['CommonDrugName'] != 'Clinical study drug')]
)

In [105]:
row_ID(med_admin_win)

(23044, 926)

In [106]:
med_admin_win.DrugCategory.value_counts()

pain agent                      4666
solution-fluid                  3867
hematological agent             2686
antineoplastic                  2333
anti-infective                  2233
antiemetic                      1825
anesthetic                      1666
steroid                         1196
antihyperglycemic                851
proton pump inhibitor            786
antidepressant                   227
cytoprotective                   220
hormone                          218
anticholinergic                   80
gout and hyperurecemia agent      51
hyperglycemic                     41
bone therapy agent (bta)          35
endocrine                         29
sedative agent                    19
antidote and reversal agent       10
immunosuppressive                  5
Name: DrugCategory, dtype: int64

#### Antineoplastic 

In [107]:
# Select window before date of metastatic diagnosis. 
med_admin_win_chemo = (
    med_admin
    [med_admin['AdministeredDate'] < med_admin['met_date']]
)

In [108]:
(
    med_admin_win_chemo.query('DrugCategory == "antineoplastic"').DrugName.value_counts().head(20)
)

fluorouracil              16526
oxaliplatin                6983
leucovorin calcium         6330
levoleucovorin calcium     2229
irinotecan hcl              680
bevacizumab                 599
carboplatin                 162
carfilzomib                 146
trastuzumab                 130
paclitaxel                  123
pembrolizumab               123
capecitabine                 91
rituximab                    85
docetaxel                    82
levoleucovorin               78
cyclophosphamide             70
daratumumab                  63
cetuximab                    63
Clinical study drug          60
panitumumab                  56
Name: DrugName, dtype: int64

In [109]:
med_admin_win_chemo.loc[:, 'adjuv'] = (
    np.where((med_admin_win_chemo['DrugName'] == 'fluorouracil') | 
             (med_admin_win_chemo['DrugName'] == 'oxaliplatin') | 
             (med_admin_win_chemo['DrugName'] == 'capecitabine'), 1, 0)
)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [110]:
med_admin_adjuv = (
    med_admin_win_chemo
    .query('adjuv == 1')
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    [['PatientID', 'adjuv']]
)

#### Antiemetic

**No indicator variable created.** 

#### Solution-fluid

**No indicator variable created.** 

#### Steroid

In [111]:
med_admin_win.loc[:, 'steroid_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'steroid') & 
             ((med_admin_win['Route'] == 'Intravenous') | 
              (med_admin_win['Route'] == 'Oral') | 
              (med_admin_win['Route'] == 'Intrajejunal')), 1, 0)
)

#### Pain

##### Opioid PO

In [112]:
# List of avialable opioids in the US. 
opioid_list = [
    'buprenorphine',
    'codeine',
    'fentanyl',
    'hydrocodone',
    'hydromorphone',
    'methadone',
    'morphine',
    'oxycodone',
    'oxymorphone',
    'tapentadol',
    'tramadol'
]

In [113]:
med_admin_win.loc[:, 'opioid_PO_diag'] = (
    np.where(((med_admin_win['Route'] == 'Oral') | 
              (med_admin_win['Route'] == 'Transdermal') | 
              (med_admin_win['Route'] == 'Sublingual')) &
             (med_admin_win['CommonDrugName'].str.contains('|'.join(opioid_list))), 1, 0)
)

##### Nonopioid PO

In [114]:
med_admin_win.loc[:, 'nonopioid_PO_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'pain agent') & 
             (med_admin_win['Route'] == 'Oral') & 
             (~med_admin_win['CommonDrugName'].str.contains('|'.join(opioid_list))), 1, 0)
)

##### Pain IV

In [115]:
med_admin_win.loc[:, 'pain_IV_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'pain agent') & 
             (med_admin_win['Route'] == 'Intravenous'), 1, 0)
)

#### Hematologic agent

##### Heparin and other parenteral agents

In [116]:
med_admin_win.loc[:, 'heparin_diag'] = (
    np.where(((med_admin_win['CommonDrugName'].str.contains('heparin')) & 
              (med_admin_win['AdministeredUnits'] == 'unit/kg/hr')) | 
             (med_admin_win['CommonDrugName'].str.contains('bivalirudin')) | 
             (med_admin_win['CommonDrugName'].str.contains('argatroban')), 1, 0)
)

###### Enoxaparin and other subcutaneous agents 

In [117]:
med_admin_win.loc[:, 'enoxaparin_diag'] = (
    np.where(((med_admin_win['CommonDrugName'].str.contains('enoxaparin')) & 
              (med_admin_win['AdministeredAmount'] > 40)) | 
             ((med_admin_win['CommonDrugName'].str.contains('dalteparin')) & 
              (med_admin_win['AdministeredAmount'] > 5000)) | 
             ((med_admin_win['CommonDrugName'].str.contains('fondaparinux')) & 
              (med_admin_win['AdministeredAmount'] > 2.5)), 1, 0)
)

##### DOAC

In [118]:
med_admin_win.loc[:, 'doac_diag'] = (
    np.where((med_admin_win['CommonDrugName'].str.contains('apixaban')) | 
             (med_admin_win['CommonDrugName'].str.contains('rivaroxaban')) | 
             (med_admin_win['CommonDrugName'].str.contains('dabigatran')) | 
             (med_admin_win['CommonDrugName'].str.contains('edoxaban')), 1, 0)
)

##### Warfarin

In [119]:
med_admin_win.loc[:, 'warfarin_diag'] = np.where((med_admin_win['CommonDrugName'].str.contains('warfarin')), 1, 0)

##### Anticoagulation merge 

In [120]:
# Combine heparin, enoxparin, DOAC, and warfarin columns into a single anticoagulation indicator variable. 
med_admin_win['ac_diag'] = (
    med_admin_win['heparin_diag'] + med_admin_win['enoxaparin_diag'] + med_admin_win['doac_diag'] + med_admin_win['warfarin_diag']
)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [121]:
# Drop heparin, enoxaparin, DOAC, and warfarin columns. 
med_admin_win = med_admin_win.drop(columns = ['heparin_diag', 'enoxaparin_diag', 'doac_diag', 'warfarin_diag'])

#### Anti-infective 

##### Anti-infective IV

In [122]:
med_admin_win.loc[:, 'antiinfective_IV_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'anti-infective') & 
             (med_admin_win['Route'] == 'Intravenous'), 1, 0)
)

##### Anti-infective PO

In [123]:
med_admin_win.loc[:, 'antiinfective_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'anti-infective') & 
             (med_admin_win['Route'] == 'Oral'), 1, 0)
)

#### Anesthetic

**No indicator variable created.** 

#### Cytoprotective

**No indicator variable created.** 

#### Antihyperglycemic

In [124]:
med_admin_win.loc[:, 'antihyperglycemic_diag'] = np.where(med_admin_win['DrugCategory'] == 'antihyperglycemic', 1, 0)

#### Proton pump inhibitor

In [125]:
med_admin_win.loc[:, 'ppi_diag'] = np.where(med_admin_win['DrugCategory'] == 'proton pump inhibitor', 1, 0)

#### Antidepressant

In [126]:
med_admin_win.loc[:, 'antidepressant_diag'] = np.where(med_admin_win['DrugCategory'] == 'antidepressant', 1, 0)

#### Bone therapy agent

In [127]:
med_admin_win.loc[:, 'bta_diag'] = np.where(med_admin_win['DrugCategory'] == 'bone therapy agent (bta)', 1, 0)

#### Hormone

In [128]:
med_admin_win.loc[:, 'thyroid_diag'] = np.where(med_admin_win['CommonDrugName'] == 'levothyroxine', 1, 0)

#### Gout and hyperurecemia agent 

**No indicator variable created.** 

#### 4.16 Immunosuppressive 

In [129]:
med_admin_win.loc[:, 'is_diag'] = np.where(med_admin_win['DrugCategory'] == 'immunosuppressive', 1, 0)

#### Sedative agent

**No indicator variable created.** 

#### Endocrine

**No indicator variable created.** 

#### Antidote and reversal agent

**No indicator variable created.** 

#### Hyperglycemic

**No indicator variable created.** 

#### Antithyroid agent

**No indicator variable created.** 

#### Anticholinergic

**No indicator variable created.** 

#### Calciumimetic

**No indicator variable created.** 

#### Targeted therapy

**No indicator variable created.** 

#### Condensing

In [130]:
# Select columns with indicator variables and PatientID, then collapse rows by PatientID and sum columns. 
med_admin_wide = (
    med_admin_win
    [med_admin_win.columns[med_admin_win.columns.str.contains('diag|PatientID')]]
    .groupby('PatientID').sum()
)

In [131]:
# Replace numbers greater than 1 with 1; 0 remains unchanged. 
med_admin_wide = (
    med_admin_wide.mask(med_admin_wide > 1, 1)
    .reset_index()
)

In [132]:
row_ID(med_admin_wide)

(926, 926)

In [133]:
# Append missing training IDs.
med_admin_wide = (
    med_admin_wide.append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(med_admin_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False
    )
    .fillna(0)
)

In [134]:
row_ID(med_admin_wide)

(6863, 6863)

In [135]:
med_admin_wide = pd.merge(med_admin_wide, med_admin_adjuv, on = 'PatientID', how = 'left').fillna(0)

In [136]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
demographics          DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
enhanced_met          DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
line_therapy          DataFrame               PatientID     <...>n[12333 rows x 9 columns]
line_therapy_1        DataFrame               PatientID     <...>\n[5155 rows x 9 columns]
med_admin             DataFrame                PatientID    <...>881806 rows x 15 columns]
med_admin_adjuv       DataFrame                PatientID  ad<...>\n[1010 rows x 2 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[6863 rows x 15 columns]
med_admin_win         DataFrame                PatientID    <...>[23044 rows x 28 columns]
med_admin_win_chemo   DataFrame                PatientID    <...>[98188 rows x 16 columns]
mortality             DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]


In [137]:
# Keep demographics, enhanced_met, med_admin_wide, med_admin_wide, and mortality
del line_therapy
del line_therapy_1
del med_admin
del med_admin_adjuv
del med_admin_win
del med_admin_win_chemo

### 5. Enhanced_MetCRCBiomarkers

In [138]:
biomarkers = pd.read_csv('Enhanced_MetCRCBiomarkers.csv')

In [139]:
biomarkers = biomarkers[biomarkers['PatientID'].isin(test_IDs)]

In [140]:
biomarkers.shape

(24757, 15)

In [141]:
biomarkers.loc[:, 'ResultDate'] = pd.to_datetime(biomarkers['ResultDate'])

**The Biomarkers dataframe is in a long format. The goal is to build a single-row-per-patient dataframe with columns reflecting a patient's biomarker status within a predefined elgibility window. For this project, the elgibility window is defined as negative infinity to +30 days from time of diagnosis of metastatic disease (ie., index date).** 

**Regarding biomarker date information, result date is the date the biomarker result was first reported, and so represents the date on which the clinician would be expected to have information about the patient’s biomarker status to inform the course of treatment. Flatiron recommends using result date as the relevant biomarker test date and using specimen received date as the proxy when result date is not available. The gaps between collected date and either received or result date are substantially more variable.**

**We'll begin by imputing specimen received date when result date is missing. Then, we'll select all biomarkers that fall within the elbility window.**

In [142]:
biomarkers.loc[:, 'SpecimenReceivedDate'] = pd.to_datetime(biomarkers['SpecimenReceivedDate'])

In [143]:
# Replace missing result date with specimen received date. 
biomarkers.loc[:, 'result_date'] = (
    np.where(biomarkers['ResultDate'].isna(), biomarkers['SpecimenReceivedDate'], biomarkers['ResultDate'])
)

In [144]:
biomarkers = pd.merge(biomarkers, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [145]:
# Create new variable that captures difference in days between result date and metastatic diagnosis. 
biomarkers.loc[:, 'bio_date_diff'] = (biomarkers['result_date'] - biomarkers['met_date']).dt.days

In [146]:
# Select all patients with biomarkers < +30 from metastatic diagnosis. 
biomarker_win = biomarkers[biomarkers['bio_date_diff'] <= 30]

**The next step is defining positive and negative staus for each biomarker. Patients with at least one confirmed positive test result for the biomarker of interest within the eligibility window will be considered “ever-positive”. This will include patients who may have confirmed negative results before and/or after a positive result within the eligibility window. A patient with an "ever-positive" biomarker during the elgibility window will have that biomarker labeled as positive.**

**In contrast, patients with at least one confirmed negative test result for the biomarker of interest, and no confirmed positive test results for the same biomarker within the eligibility window may be considered “only-negative”. A patient with an "only-negative" biomarker during the elgibility window will have that biomarker labeled as negative.**

**Lastly, if the biomarker is neither positive or negative, then it will be labeled as unknown.**

In [147]:
# Create indicator variable where where 2 if positive, 1 if negative, and 0 if unknown or missing. 
conditions = [
    (biomarker_win['BiomarkerStatus'] == 'Mutation positive') | 
    (biomarker_win['BiomarkerStatus'] == 'Loss of MMR protein expression (MMR protein deficiency found)') | 
    (biomarker_win['BiomarkerStatus'] == 'MSI-H'), 
    (biomarker_win['BiomarkerStatus'] == 'Mutation negative') | 
    (biomarker_win['BiomarkerStatus'] == 'Normal MMR protein expression (No loss of nuclear expression of MMR protein)') |
    (biomarker_win['BiomarkerStatus'] == 'MSS') | 
    (biomarker_win['BiomarkerStatus'] == 'MSI-L') |
    (biomarker_win['BiomarkerStatus'] == 'MSS-Ambiguous') |
    (biomarker_win['BiomarkerStatus'] == 'Equivocal')
]

choices = [2,1]
biomarker_win.loc[:, 'bio_status'] = np.select(conditions, choices, default = 0)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [148]:
# Select highest number biomarker status among duplicates, merge with nonduplciates, then pivot. 
biomarker_wide = (
    biomarker_win
    .sort_values(by = ['PatientID', 'BiomarkerName','bio_status'], ascending = False)
    .drop_duplicates(subset = ['PatientID', 'BiomarkerName'], keep = 'first')
    .pivot(index = 'PatientID', columns = 'BiomarkerName', values = 'bio_status')
    .reset_index()
)
biomarker_wide.columns.name = None
biomarker_wide = biomarker_wide.rename(columns = {'MMR/MSI': 'dMMR_MSIh'})

In [149]:
row_ID(biomarker_wide)

(3749, 3749)

In [150]:
biomarker_wide = (
    biomarker_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(biomarker_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [151]:
row_ID(biomarker_wide)

(6863, 6863)

In [152]:
biomarker_wide['BRAF'] = (
    biomarker_wide['BRAF'].replace({
        2: 'mutated',
        1: 'wild-type',
        0: 'unknown',
        np.nan: 'unknown'})
)

In [153]:
biomarker_wide['KRAS'] = (
    biomarker_wide['KRAS'].replace({
        2: 'mutated',
        1: 'wild-type',
        0: 'unknown',
        np.nan: 'unknown'})
)

In [154]:
biomarker_wide['dMMR_MSIh'] = (
    biomarker_wide['dMMR_MSIh'].replace({
        2: 'yes',
        1: 'no',
        0: 'unknown',
        np.nan: 'unknown'})
)

In [155]:
biomarker_wide['NRAS'] = (
    biomarker_wide['NRAS'].replace({
        2: 'mutated',
        1: 'wild-type',
        0: 'unknown',
        np.nan: 'unknown'})
)

In [156]:
# IDs of patients with mutation positive BRAF that is V600E. 
v600e_id = (
    biomarker_win
    .query('BiomarkerName == "BRAF"')
    .query('BiomarkerStatus == "Mutation positive"')
    .query('BiomarkerDetail == "V600E BRAF mutation"')
    .PatientID
)

In [157]:
# Identify patients with V600E mutated BRAF vs. other. 
conditions = [
    (biomarker_wide['PatientID'].isin(v600e_id)),
    (~biomarker_wide['PatientID'].isin(v600e_id)) & (biomarker_wide['BRAF'] == 'mutated')
]

choices = ['mutated V600E', 'mutated other']
biomarker_wide.loc[:, 'BRAF_n'] = np.select(conditions, choices, default = biomarker_wide['BRAF'])

In [158]:
biomarker_wide = (
    biomarker_wide
    .drop(columns = ['BRAF'])
    .rename(columns = {'BRAF_n': 'BRAF'}))

In [159]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
biomarker_wide   DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
biomarker_win    DataFrame               PatientID Biom<...>n[8849 rows x 19 columns]
biomarkers       DataFrame               PatientID Biom<...>[24757 rows x 18 columns]
demographics     DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
enhanced_met     DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
med_admin_wide   DataFrame              PatientID  ster<...>n[6863 rows x 15 columns]
mortality        DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]


In [160]:
# Keep biomarker_wide, demographics, enhanced_met, med_admin_wide, and mortality
del biomarker_win
del biomarkers

### 6. Insurance

In [161]:
insurance = pd.read_csv('Insurance.csv')

In [162]:
insurance = insurance[insurance['PatientID'].isin(test_IDs)]

In [163]:
row_ID(insurance)

(22911, 6552)

**The insurance table contains patient insurance/payer information. Patients may have multiple payer categories concurrently. Start date is populated roughly 80% of the time, while end date is populated about 20% of the time. This mutiple-row-per-patient table will be transformed into a single-row-per-patient table. Indicator variables for each payer category active at time of metastatic diagnosis will be made as columns. Insurance will be considered active if start date is less than 30 days from advanced diagnosis regardless of end date.** 

In [164]:
insurance.loc[:, 'StartDate'] = pd.to_datetime(insurance['StartDate'])

In [165]:
insurance = pd.merge(insurance, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [166]:
# Remove years with start dates less than 1900 which is likely a coding error. 
insurance = insurance[(insurance['StartDate']).dt.year >= 1900]

In [167]:
insurance.loc[:, 'insurance_date_diff'] = (insurance['StartDate'] - insurance['met_date']).dt.days

In [168]:
insurance_win = insurance[insurance['insurance_date_diff'] <= 30]

In [169]:
row_ID(insurance)

(20105, 6076)

In [170]:
# Recode payer category 
conditions = [
    (insurance_win['IsMedicareAdv'] == 'Yes') | 
    (insurance_win['IsPartAOnly'] == 'Yes') | 
    (insurance_win['IsPartBOnly'] == 'Yes') |
    (insurance_win['IsPartAandPartB'] == 'Yes') |
    (insurance_win['IsPartDOnly'] == 'Yes'),
    (insurance_win['IsManagedGovtPlan'] == 'Yes'),
    (insurance_win['IsManagedMedicaid'] == 'Yes'),
    (insurance_win['IsMedicareMedicaid'] == 'Yes')]

choices = ['Medicare', 'Other Government Program', 'Medicaid', 'medicare_medicaid']

insurance_win.loc[:, 'payer_category'] = np.select(conditions, choices, insurance_win['PayerCategory'])

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


#### Medicare

In [171]:
insurance_win.loc[:, 'medicare'] = np.where(insurance_win['payer_category'] == 'Medicare', 1, 0)

#### Medicaid

In [172]:
insurance_win.loc[:, 'medicaid'] = np.where(insurance_win['payer_category'] == 'Medicaid', 1, 0)

#### Medicare/Medicaid 

In [173]:
insurance_win.loc[:, 'medicare_medicaid'] = np.where(insurance_win['payer_category'] == 'medicare_medicaid', 1, 0)

#### Commercial 

In [174]:
insurance_win.loc[:, 'commercial'] = np.where(insurance_win['payer_category'] == 'Commercial Health Plan', 1, 0)

#### Patient Assistance Programs 

In [175]:
insurance_win.loc[:, 'patient_assistance'] = np.where(insurance_win['payer_category'] == 'Patient Assistance Program', 1, 0)

#### Other Government Program 

In [176]:
insurance_win.loc[:, 'other_govt'] = np.where(insurance_win['payer_category'] == 'Other Government Program', 1, 0)

#### Self Pay 

In [177]:
insurance_win.loc[:, 'self_pay'] = np.where(insurance_win['payer_category'] == 'Self Pay', 1, 0)

#### Other Payer

In [178]:
insurance_win.loc[:, 'other'] = np.where(insurance_win['payer_category'] == 'Other Payer - Type Unknown', 1, 0)

#### Condense 

In [179]:
# After dropping 'insurance_date_diff', add columns by PatientID.
insurance_wide = (
    insurance_win
    .drop(columns = ['insurance_date_diff'])
    .groupby('PatientID').sum()
)

In [180]:
# Set any value greater than 1 to 1; leave 0 unchanged. 
insurance_wide = (
    insurance_wide
    .mask(insurance_wide > 1, 1)
    .reset_index()
)

In [181]:
row_ID(insurance_wide)

(4968, 4968)

In [182]:
# Append missing training IDs.
insurance_wide = (
    insurance_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(insurance_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [183]:
row_ID(insurance_wide)

(6863, 6863)

In [184]:
insurance_wide = insurance_wide.fillna(0)

In [185]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
biomarker_wide   DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
demographics     DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
enhanced_met     DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
insurance        DataFrame               PatientID     <...>[20105 rows x 16 columns]
insurance_wide   DataFrame              PatientID  medi<...>\n[6863 rows x 9 columns]
insurance_win    DataFrame               PatientID     <...>n[9687 rows x 25 columns]
med_admin_wide   DataFrame              PatientID  ster<...>n[6863 rows x 15 columns]
mortality        DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]


In [186]:
# Keep biomarker_wide, demographics, enhanced_met, insurance_wide, med_admin_wide, and mortality
del insurance
del insurance_win

### 7. ECOG

In [187]:
ecog = pd.read_csv('ECOG.csv')

In [188]:
ecog = ecog[ecog['PatientID'].isin(test_IDs)]

In [189]:
row_ID(ecog)

(138410, 5261)

**The ECOG table is a longitudinal record of structured ECOG scores captured in the EHR for each patient. Many patients have multiple ECOG scores reported. A new dataframe will be built where one ECOG score will be assigned to each patient. The index date will be date of advanced diagnosis with an elgible window period of +30 days to -90 days from advanced diagnosis. The ECOG score closest to index date will be assigned to the patient. In the case of two ECOG scores on the same day or equidistant but on opposite sides of the index date, the higher ECOG score (worse performance) will be selected.** 

**BaselineECOG is a composite table that selects one ECOG score within +7 days and -30 days of a line of therapy. Patients might have two baseline ECOG values for line number 1 due to maintenance therapy. BaselineECOG will not be used for creating baseline models.** 

In [190]:
ecog = pd.merge(ecog, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [191]:
ecog.loc[:, 'EcogDate'] = pd.to_datetime(ecog['EcogDate'])      

In [192]:
ecog.loc[:, 'ecog_date_diff'] = (ecog['EcogDate'] - ecog['met_date']).dt.days

In [193]:
ecog_win = ecog[(ecog['ecog_date_diff'] >= -90) & (ecog['ecog_date_diff'] <= 30)]

In [194]:
row_ID(ecog_win)

(8350, 3190)

In [195]:
# Time from metastatic diagnosis to ECOG date will be converted to an absolute value. 
ecog_win.loc[:, 'ecog_date_diff'] = ecog_win['ecog_date_diff'].abs()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [196]:
# Sort values with ECOG nearest to time of diagnosis as top row (and largest ECOG if multiple ECOGs that day) then select top row.   ECOG date nearest to day of diagnosis as top row and largest ES
ecog_diagnosis_wide = (
    ecog_win
    .sort_values(by = ['PatientID', 'ecog_date_diff', 'EcogValue'], ascending = [True, True, False])
    .drop_duplicates(subset = ['PatientID'], keep = 'first' )
    .filter(items = ['PatientID', 'EcogValue'])
    .rename(columns = {'EcogValue': 'ecog_diagnosis'})
)

In [197]:
row_ID(ecog_diagnosis_wide)

(3190, 3190)

In [198]:
# Append missing training IDs. 
ecog_diagnosis_wide = (
    ecog_diagnosis_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(ecog_diagnosis_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
    .fillna('unknown')
)

In [199]:
row_ID(ecog_diagnosis_wide)

(6863, 6863)

In [200]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
demographics          DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
ecog                  DataFrame                PatientID    <...>[138410 rows x 6 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6863 rows x 2 columns]
ecog_win              DataFrame                PatientID    <...>\n[8350 rows x 6 columns]
enhanced_met          DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[6863 rows x 9 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[6863 rows x 15 columns]
mortality             DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]


In [201]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, insurance_wide, med_admin_wide, and mortality
del ecog
del ecog_win

### 8. Vitals

In [202]:
vitals = pd.read_csv('Vitals.csv')

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [203]:
vitals = vitals[vitals['PatientID'].isin(test_IDs)]

In [204]:
row_ID(vitals)

(2146716, 6851)

**The Vitals table is a longitudinal record of vitals captured in the EHR for each patient. A weight and BMI variable at time of advanced diagnosis will be created. The elgibility window will be -90 days to +30 days from advanced diagnosis. Average height from all visits will be used to calculate BMI. In the case of two weights on the same day or equidistant but on opposite sides of the index date, the lowest weight will be selected. Percent change in weight and weight slope 3 months within advanced diagnosis will be calculated as in the LCPI model. Patients must have at least two weight recordings to calculate percent change in weight or weight slope.** 

#### Weight and BMI

In [205]:
# Create weight dataframe; remove weight values that are empty or equal to zero.
weight = (
    vitals
    .query('Test == "body weight"')
    .filter(items = ['PatientID', 'TestDate', 'TestResultCleaned'])
    .rename(columns = {'TestResultCleaned': 'weight'})
    .dropna(subset = ['weight'])
    .query('weight != 0')
)

In [206]:
weight.loc[:, 'TestDate'] = pd.to_datetime(weight['TestDate'])

In [207]:
weight = pd.merge(weight, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [208]:
# Weight elgibliity window is -90 and +30 from metastatic diagnosis diagnosis. 
weight_win_bmi = (
    weight
    .assign(weight_date_diff = (weight['TestDate'] - weight['met_date']).dt.days)
    .query('weight_date_diff >= -90 and weight_date_diff <= 30')
)

In [209]:
weight_win_bmi.loc[:, 'weight_date_diff'] = weight_win_bmi['weight_date_diff'].abs()

In [210]:
# Select weight closest to date of metastatic diagnosis; lowest weight selected in the event of two weights on same day or equidistant. 
weight_bmi_wide = (
    weight_win_bmi
    .sort_values(by = ['PatientID', 'weight_date_diff', 'weight'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'weight'])
    .rename(columns = {'weight': 'weight_diag'})
)

In [211]:
# Dataframe of average height for each patient. 
height_avg = (
    vitals
    .query('Test == "body height"')
    .filter(items = ['PatientID', 'TestResultCleaned'])
    .groupby('PatientID')['TestResultCleaned'].mean()
    .to_frame()
    .reset_index()
    .rename(columns = {'TestResultCleaned': 'height_avg'})
)

In [212]:
weight_bmi_wide = pd.merge(weight_bmi_wide, height_avg, on = 'PatientID', how = 'left')

In [213]:
# Create BMI column. 
weight_bmi_wide = (
    weight_bmi_wide
    .assign(bmi_diag = lambda x: (x['weight_diag']/(x['height_avg']*x['height_avg']))*10000)
    .drop(columns = ['height_avg'])
)

In [214]:
# Append excluded IDs from training set and create a missing variable for those without BMI at diagnosis. 
weight_bmi_wide = (
    weight_bmi_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(weight_bmi_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [215]:
row_ID(weight_bmi_wide)

(6863, 6863)

In [216]:
weight_bmi_wide.loc[:, 'bmi_diag_na'] = np.where(weight_bmi_wide['bmi_diag'].isna(), 1, 0)

#### Percent change 

In [217]:
# Select elgbility window of -90 to +90 days from advanced diagnosis.
weight_win_summary = (
    weight
    .assign(weight_date_diff = (weight['TestDate'] - weight['met_date']).dt.days)
    .query('weight_date_diff >= -90 and weight_date_diff <= 90')
)

In [218]:
# Select patients with more than 1 weight recording within elgibility window.
weight_win_summary = weight_win_summary[weight_win_summary.duplicated(subset = ['PatientID'], keep = False)]

In [219]:
# Select weight from the earliest time within elgibility window. 
weight_tmin = weight_win_summary.loc[weight_win_summary.groupby('PatientID')['weight_date_diff'].idxmin()]

In [220]:
# Select weight from the latest time within elgibility window. 
weight_tmax = weight_win_summary.loc[weight_win_summary.groupby('PatientID')['weight_date_diff'].idxmax()]

In [221]:
# Combine above two dataframes and sort from earliest recorded weight to latest recorded weight for each patient. 
weight_tcomb = (
    pd.concat([weight_tmin, weight_tmax])
    .sort_values(by = ['PatientID', 'weight_date_diff'], ascending = True)
)

In [222]:
row_ID(weight_tcomb)

(10826, 5413)

In [223]:
weight_tcomb.loc[:, 'weight_pct_change'] = weight_tcomb.groupby('PatientID')['weight'].pct_change()

In [224]:
weight_tcomb.loc[:, 'diff_date_diff'] = weight_tcomb['weight_date_diff'].diff()

In [225]:
# Drop empty rows for weight_pct_change.
weight_pct_wide = (
    weight_tcomb
    .dropna(subset = ['weight_pct_change'])
    .filter(items = ['PatientID', 'weight_pct_change', 'diff_date_diff'])
)

In [226]:
row_ID(weight_pct_wide)

(5413, 5413)

In [227]:
# Append missing training IDs and create a missing variable for those without weight_pct_change. 
weight_pct_wide = (
    weight_pct_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(weight_pct_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
    .drop(columns = ['diff_date_diff'])
)

In [228]:
row_ID(weight_pct_wide)

(6863, 6863)

In [229]:
weight_pct_wide.loc[:, 'weight_pct_na'] = np.where(weight_pct_wide['weight_pct_change'].isna(), 1, 0)

#### Weight slope

In [230]:
from scipy.stats import linregress 

In [231]:
weight_win_summary.loc[:, 'date_ordinal'] = weight_win_summary['TestDate'].map(dt.datetime.toordinal)

In [232]:
# Dataframe of slope for weight recordings within window period (kg/day).
weight_slope_wide = (
    weight_win_summary
    .groupby('PatientID')
    .apply(lambda x: pd.Series(linregress(x['date_ordinal'], x['weight'])))
    .rename(columns = {0: 'weight_slope'})
    .reset_index()
    .filter(items = ['PatientID', 'weight_slope']))   

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm


In [233]:
row_ID(weight_slope_wide)

(5413, 5413)

In [234]:
# Append missing training IDs. 
weight_slope_wide = (
    weight_slope_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(weight_slope_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [235]:
row_ID(weight_slope_wide)

(6863, 6863)

#### Weight merge 

In [236]:
weight_wide = pd.merge(weight_bmi_wide, weight_pct_wide, on = 'PatientID')

In [237]:
weight_wide = pd.merge(weight_wide, weight_slope_wide, on = 'PatientID')

In [238]:
row_ID(weight_wide)

(6863, 6863)

In [239]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
demographics          DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6863 rows x 2 columns]
enhanced_met          DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
height_avg            DataFrame              PatientID  heig<...>\n[6810 rows x 2 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[6863 rows x 9 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[6863 rows x 15 columns]
mortality             DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]
vitals                DataFrame                  PatientID  <...>146716 rows x 16 columns]
weight                DataFrame                PatientID   T<...>[271931 rows x 4 columns]


In [240]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, insurance_wide, med_admin_wide, mortality, 
# and weight_wide
del height_avg
del vitals
del weight
del weight_bmi_wide
del weight_pct_wide
del weight_slope_wide
del weight_tcomb
del weight_tmax
del weight_tmin
del weight_win_bmi
del weight_win_summary

### Labs

In [241]:
lab = pd.read_csv('Lab.csv')

In [242]:
lab = lab[lab['PatientID'].isin(test_IDs)]

In [243]:
row_ID(lab)

(6178543, 6629)

**The Lab table is a longitudinal record of lab captured in the EHR with multiple-rows-per-patient. A single-patient-per-row table will be built focusing on the following NCCN recommended labs:** 
* **Creatinine -- (LOINC: 2160-0 and 38483-4)**
* **Hemoglobin -- (LOINC: 718-7 and 20509-6)**
* **White blood cell count -- (LOINC: 26464-8 and 6690-2)**
* **Neutrophil count -- (LOINC: 26499-4, 751-8, 30451-9, and 753-4)**
* **Albumin, serum -- (LOINC: 1751-7)**
* **Total bilirubin -- (LOINC: 42719-5 and 1975-2)**
* **Sodium — (LOINC: 2947-0 and 2951-2)**
* **Bicarb — (LOINC: 1963-8, 1959-6, 14627-4, 1960-4, and 2028-9)**
* **Calcium — (LOINC: 17861-6 and 49765-1)** 
* **AST — (LOINC: 1920-8)**
* **ALT — (LOINC: 1742-6, 1743-4, and 1744-2)**
* **Platelet -- (LOINC: 26515-7, 777-3, 778-1, and 49497-1)**
* **Potassium -- (LOINC: 6298-4 and 2823-3)**
* **Chloride -- (LOINC: 2075-0)**
* **BUN -- (LOINC: 3094-0)**
* **ALP -- (LOINC: 6768-6)**
* **CEA -- (LOINC: 2039-6)**

**The index date will be time of advanced diagnosis with an elgibility window of -90 days to +30 days. The lab value closest to the index date will be selected for each patient. The following summary statistics, using an elgibility window of negative infinity to +30 days from advanced diagnosis, will also be created for the above variables:** 
* **Max**
* **Min**
* **Mean**
* **Standard deviation** 
* **Slope**

#### 9.1 Baseline lab values

In [244]:
lab = pd.merge(lab, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [245]:
lab.loc[:, 'ResultDate'] = pd.to_datetime(lab['ResultDate']) 

In [246]:
# Select rows with clinically relevant labs.
lab_core = (
    lab[
    (lab['LOINC'] == "2160-0") |
    (lab['LOINC'] == "38483-4") | 
    (lab['LOINC'] == "718-7") |
    (lab['LOINC'] == "20509-6") |
    (lab['LOINC'] == "26464-8") |
    (lab['LOINC'] == "6690-2") |
    (lab['LOINC'] == "26499-4") |
    (lab['LOINC'] == "751-8") |
    (lab['LOINC'] == "30451-9") |
    (lab['LOINC'] == "753-4") |
    (lab['LOINC'] == "1751-7") |
    (lab['LOINC'] == "42719-5") |
    (lab['LOINC'] == "1975-2") |
    (lab['LOINC'] == "2947-0") |
    (lab['LOINC'] == "2951-2") |
    (lab['LOINC'] == "1963-8") |
    (lab['LOINC'] == "1959-6") |
    (lab['LOINC'] == "14627-4") |
    (lab['LOINC'] == "1960-4") |
    (lab['LOINC'] == "2028-9") |
    (lab['LOINC'] == "17861-6") |
    (lab['LOINC'] == "49765-1") |
    (lab['LOINC'] == "1920-8") |
    (lab['LOINC'] == "1742-6") | 
    (lab['LOINC'] == "1743-4") |
    (lab['LOINC'] == "1744-2") |
    (lab['LOINC'] == "26515-7") | 
    (lab['LOINC'] == "777-3") |
    (lab['LOINC'] == "778-1") |
    (lab['LOINC'] == "49497-1") | 
    (lab['LOINC'] == "6298-4") |
    (lab['LOINC'] == "2823-3") |
    (lab['LOINC'] == "2075-0") | 
    (lab['LOINC'] == "3094-0") |
    (lab['LOINC'] == "6768-6") |
    (lab['LOINC'] == "2039-6")]
    .filter(items = ['PatientID', 
                     'ResultDate', 
                     'LOINC', 
                     'LabComponent', 
                     'TestUnits', 
                     'TestUnitsCleaned', 
                     'TestResult', 
                     'TestResultCleaned', 
                     'met_date'])
)

In [247]:
conditions = [
    ((lab_core['LOINC'] == '2160-0') | (lab_core['LOINC'] == '38483-4')),
    ((lab_core['LOINC'] == '718-7') | (lab_core['LOINC'] == '20509-6')),
    ((lab_core['LOINC'] == '26464-8') | (lab_core['LOINC'] == '6690-2')), 
    ((lab_core['LOINC'] == '26499-4') | (lab_core['LOINC'] == '751-8') | (lab_core['LOINC'] == '30451-9') | (lab_core['LOINC'] == '753-4')),
    (lab_core['LOINC'] == '1751-7'),
    ((lab_core['LOINC'] == '42719-5') | (lab_core['LOINC'] == '1975-2')),
    ((lab_core['LOINC'] == '2947-0') | (lab_core['LOINC'] == '2951-2')),
    ((lab_core['LOINC'] == '1963-8') | (lab_core['LOINC'] == '1959-6') | (lab_core['LOINC'] == '14627-4') | (lab_core['LOINC'] == '1960-4') | (lab_core['LOINC'] == '2028-9')),
    ((lab_core['LOINC'] == '17861-6') | (lab_core['LOINC'] == '49765-1')),
    (lab_core['LOINC'] == '1920-8'),
    ((lab_core['LOINC'] == '1742-6') | (lab_core['LOINC'] == '1743-4') | (lab_core['LOINC'] == '1744-2')),
    ((lab_core['LOINC'] == '26515-7') | (lab_core['LOINC'] == '777-3') | (lab_core['LOINC'] == '778-1') | (lab_core['LOINC'] == '49497-1')),
    ((lab_core['LOINC'] == '6298-4') | (lab_core['LOINC'] == '2823-3')),
    (lab_core['LOINC'] == '2075-0'), 
    (lab_core['LOINC'] == '3094-0'),
    (lab_core['LOINC'] == '6768-6'),
    (lab_core['LOINC'] == '2039-6')]

choices = ['creatinine', 
           'hemoglobin', 
           'wbc', 
           'neutrophil_count',  
           'albumin', 
           'total_bilirubin', 
           'sodium', 
           'bicarb',
           'calcium',
           'ast', 
           'alt',
           'platelet',
           'potassium', 
           'chloride',
           'bun',
           'alp',
           'cea']

lab_core.loc[:, 'lab_name'] = np.select(conditions, choices)

In [248]:
# Remove missing lab values. 
lab_core = lab_core.dropna(subset = ['TestResultCleaned'])

In [249]:
conditions = [
    ((lab_core['lab_name'] == 'wbc') | (lab_core['lab_name'] == 'neutrophil_count') | (lab_core['lab_name'] == 'platelet')) & 
    (lab_core['TestUnits'] == '10*3/L'),
    (lab_core['lab_name'] == 'hemoglobin') & (lab_core['TestUnits'] == 'g/uL')]

choices = [lab_core['TestResultCleaned'] * 1000000,
           lab_core['TestResultCleaned'] / 100000]

lab_core.loc[:, 'test_result_cleaned'] = np.select(conditions, choices, default = lab_core['TestResultCleaned'])

In [250]:
# Elgibliity window is -90 and +30 from advanced diagnosis. 
lab_core_win = (
    lab_core
    .assign(lab_date_diff = (lab_core['ResultDate'] - lab_core['met_date']).dt.days)
    .query('lab_date_diff >= -90 and lab_date_diff <= 30')
    .filter(items = ['PatientID', 'ResultDate', 'TestResultCleaned', 'lab_name', 'met_date', 'test_result_cleaned', 'lab_date_diff'])
)

In [251]:
lab_core_win.loc[:, 'lab_date_diff'] = lab_core_win['lab_date_diff'].abs()

In [252]:
# Select lab closest to date of advanced diagnosis and pivot to a wide table. 
lab_diag_wide = (
    lab_core_win
    .loc[lab_core_win.groupby(['PatientID', 'lab_name'])['lab_date_diff'].idxmin()]
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_diag',
        'creatinine': 'creatinine_diag',
        'hemoglobin': 'hemoglobin_diag',
        'neutrophil_count': 'neutrophil_count_diag',
        'total_bilirubin': 'total_bilirubin_diag',
        'wbc': 'wbc_diag',
        'sodium': 'sodium_diag', 
        'bicarb': 'bicarb_diag',
        'calcium': 'calcium_diag',
        'ast': 'ast_diag', 
        'alt': 'alt_diag',
        'platelet': 'platelet_diag',
        'potassium': 'potassium_diag',
        'chloride': 'chloride_diag',
        'bun': 'bun_diag',
        'alp': 'alp_diag', 
        'cea': 'cea_diag'})
)

lab_diag_wide.columns.name = None

In [253]:
row_ID(lab_diag_wide)

(4486, 4486)

In [254]:
lab_diag_wide = (
    lab_diag_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(lab_diag_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [255]:
row_ID(lab_diag_wide)

(6863, 6863)

In [256]:
# Create missing variables for labs at time of diagnosis. 
for x in range (1, len(lab_diag_wide.columns)):
    lab_diag_wide.loc[:, lab_diag_wide.columns[x]+'_na'] = np.where(lab_diag_wide[lab_diag_wide.columns[x]].isna(), 1, 0)

In [257]:
list(lab_diag_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'cea_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'cea_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na']

#### Mean, max, min, and standard deviation

In [258]:
# Elgibility window is negative infinity to +30 from advanced diagnosis. 
lab_core_win_summ = (
    lab_core
    .assign(lab_date_diff = (lab_core['ResultDate'] - lab_core['met_date']).dt.days)
    .query('lab_date_diff <= 30')
    .filter(items = ['PatientID', 'ResultDate', 'TestResultCleaned', 'lab_name', 'met_date', 'test_result_cleaned', 'lab_date_diff'])
)

In [259]:
# Pivot table of average values for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_avg_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].mean()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_avg',
        'creatinine': 'creatinine_avg',
        'hemoglobin': 'hemoglobin_avg',
        'neutrophil_count': 'neutrophil_count_avg',
        'total_bilirubin': 'total_bilirubin_avg',
        'wbc': 'wbc_avg',
        'sodium': 'sodium_avg', 
        'bicarb': 'bicarb_avg',
        'calcium': 'calcium_avg',
        'ast': 'ast_avg', 
        'alt': 'alt_avg',
        'platelet': 'platelet_avg',
        'potassium': 'potassium_avg',
        'chloride': 'chloride_avg',
        'bun': 'bun_avg',
        'alp': 'alp_avg',
        'cea': 'cea_avg'})
)

lab_avg_wide.columns.name = None

In [260]:
row_ID(lab_avg_wide)

(4728, 4728)

In [261]:
# Pivot table of maximum values for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_max_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].max()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_max',
        'creatinine': 'creatinine_max',
        'hemoglobin': 'hemoglobin_max',
        'neutrophil_count': 'neutrophil_count_max',
        'total_bilirubin': 'total_bilirubin_max',
        'wbc': 'wbc_max', 
        'sodium': 'sodium_max', 
        'bicarb': 'bicarb_max',
        'calcium': 'calcium_max',
        'ast': 'ast_max', 
        'alt': 'alt_max',
        'platelet': 'platelet_max',
        'potassium': 'potassium_max',
        'chloride': 'chloride_max',
        'bun': 'bun_max', 
        'alp': 'alp_max', 
        'cea': 'cea_max'})
)

lab_max_wide.columns.name = None

In [262]:
row_ID(lab_max_wide)

(4728, 4728)

In [263]:
# Pivot table of minimum values for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_min_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].min()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_min',
        'creatinine': 'creatinine_min',
        'hemoglobin': 'hemoglobin_min',
        'neutrophil_count': 'neutrophil_count_min',
        'total_bilirubin': 'total_bilirubin_min',
        'wbc': 'wbc_min',
        'sodium': 'sodium_min', 
        'bicarb': 'bicarb_min',
        'calcium': 'calcium_min',
        'ast': 'ast_min', 
        'alt': 'alt_min',
        'platelet': 'platelet_min',
        'potassium': 'potassium_min',
        'chloride': 'chloride_min',
        'bun': 'bun_min',
        'alp': 'alp_min',
        'cea': 'cea_min'})
)

lab_min_wide.columns.name = None

In [264]:
row_ID(lab_min_wide)

(4728, 4728)

In [265]:
# Pivot table of standard deviation for core labs during elgibility period of -90 to -30 days from advanced diagnosis. 
lab_std_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].std()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_std',
        'creatinine': 'creatinine_std',
        'hemoglobin': 'hemoglobin_std',
        'neutrophil_count': 'neutrophil_count_std',
        'total_bilirubin': 'total_bilirubin_std',
        'wbc': 'wbc_std',
        'sodium': 'sodium_std', 
        'bicarb': 'bicarb_std',
        'calcium': 'calcium_std',
        'ast': 'ast_std', 
        'alt': 'alt_std',
        'platelet': 'platelet_std',
        'potassium': 'potassium_std',
        'chloride': 'chloride_std',
        'bun': 'bun_std',
        'alp': 'alp_std',
        'cea': 'cea_std'})
)

lab_std_wide.columns.name = None

In [266]:
row_ID(lab_std_wide)

(4728, 4728)

In [267]:
lab_summary_wide = pd.merge(lab_avg_wide, lab_max_wide, on = 'PatientID', how = 'outer')

In [268]:
lab_summary_wide = pd.merge(lab_summary_wide, lab_min_wide, on = 'PatientID', how = 'outer')

In [269]:
lab_summary_wide = pd.merge(lab_summary_wide, lab_std_wide, on = 'PatientID', how = 'outer')

In [270]:
row_ID(lab_summary_wide)

(4728, 4728)

In [271]:
lab_summary_wide = (
    lab_summary_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(lab_summary_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [272]:
row_ID(lab_summary_wide)

(6863, 6863)

#### Slope

In [273]:
lab_core_win_summ.loc[:, 'result_date_ordinal'] = lab_core_win_summ['ResultDate'].map(dt.datetime.toordinal)

In [274]:
lab_slope_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])
    .apply(lambda x: pd.Series(linregress(x['result_date_ordinal'], x['test_result_cleaned'])))
    .rename(columns = {0: 'slope'})
    .reset_index()
    .filter(items = ['PatientID', 'lab_name', 'slope'])
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'slope')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_slope',
        'creatinine': 'creatinine_slope',
        'hemoglobin': 'hemoglobin_slope',
        'neutrophil_count': 'neutrophil_count_slope',
        'total_bilirubin': 'total_bilirubin_slope',
        'wbc': 'wbc_slope',
        'sodium': 'sodium_slope', 
        'bicarb': 'bicarb_slope',
        'calcium': 'calcium_slope',
        'ast': 'ast_slope', 
        'alt': 'alt_slope',
        'platelet': 'platelet_slope',
        'potassium': 'potassium_slope',
        'chloride': 'chloride_slope',
        'bun': 'bun_slope',
        'alp': 'alp_slope',
        'cea': 'cea_slope'})
)

lab_slope_wide.columns.name = None

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:184: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [275]:
row_ID(lab_slope_wide)

(4728, 4728)

In [276]:
lab_slope_wide = (
    lab_slope_wide
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(lab_slope_wide['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [277]:
# Create missing variables for lab slope. 
for x in range (1, len(lab_slope_wide.columns)):
    lab_slope_wide.loc[:, lab_slope_wide.columns[x]+'_na'] = np.where(lab_slope_wide[lab_slope_wide.columns[x]].isna(), 1, 0)

In [278]:
row_ID(lab_slope_wide)

(6863, 6863)

#### Merge

In [279]:
lab_wide = pd.merge(lab_diag_wide, lab_summary_wide, on = 'PatientID')

In [280]:
lab_wide = pd.merge(lab_wide, lab_slope_wide, on = 'PatientID')

In [281]:
row_ID(lab_wide)

(6863, 6863)

In [282]:
list(lab_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'cea_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'cea_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na',
 'albumin_avg',
 'alp_avg',
 'alt_avg',
 'ast_avg',
 'bicarb_avg',
 'bun_avg',
 'calcium_avg',
 'cea_avg',
 'chloride_avg',
 'creatinine_avg',
 'hemoglobin_avg',
 'neutrophil_count_avg',
 'platelet_avg',
 'potassium_avg',
 'sodium_avg',
 'total_bilirubin_avg',
 'wbc_avg',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarb_max',
 'bun_max',
 'c

In [283]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
demographics          DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6863 rows x 2 columns]
enhanced_met          DataFrame               PatientID diag<...>\n[6863 rows x 7 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[6863 rows x 9 columns]
lab                   DataFrame                 PatientID   <...>178543 rows x 18 columns]
lab_avg_wide          DataFrame              PatientID  albu<...>n[4728 rows x 18 columns]
lab_core              DataFrame                 PatientID Re<...>918831 rows x 11 columns]
lab_core_win          DataFrame                 PatientID Re<...>[173044 rows x 7 columns]
lab_core_win_summ     DataFrame                 PatientID Re<...>[571581 rows x 8 columns]


In [284]:
# Keep biomarker_wide, demographics, ecog_diagnosis_wide, enhanced_met, insurance_wide, lab_wide, med_admin_wide, 
# mortality, and weight_wide
del lab
del lab_avg_wide
del lab_core
del lab_core_win
del lab_core_win_summ
del lab_diag_wide
del lab_max_wide
del lab_min_wide
del lab_slope_wide
del lab_std_wide
del lab_summary_wide

### Diagnosis

In [285]:
diagnosis = pd.read_csv('Diagnosis.csv')

In [286]:
diagnosis = diagnosis[diagnosis['PatientID'].isin(test_IDs)]

In [287]:
row_ID(diagnosis)

(218887, 6863)

In [288]:
diagnosis.sample(5)

,PatientID,PracticeID,DiagnosisDate,DiagnosisCode,DiagnosisCodeSystem,DiagnosisDescription
83385,FDA39ACB31320,S0015D5ED587A,2018-12-19,C78.6,ICD-10-CM,Secondary malignant neoplasm of retroperitoneu...
591480,F169322040C24,S507DCE5DD8C6,2017-06-03,I10,ICD-10-CM,Essential (primary) hypertension
296892,F312D7F5C85E2,SCC8EE4FB66C3,2020-09-07,D46.21,ICD-10-CM,Refractory anemia with excess of blasts 1
687487,F160E3672947C,S507DCE5DD8C6,2017-07-11,N32.81,ICD-10-CM,Overactive bladder
485080,F0C5BC58E2977,SB875B0336958,2020-09-13,F17.210,ICD-10-CM,"Nicotine dependence, cigarettes, uncomplicated"


**The Diagnosis table is in long format and has close to 12,000 unique ICD 9 and 10 codes. The median number of ICD codes per patient is 8 with a standard deviation of 106 which shows the varaibility in number of ICD codes per patent..** 

**ICD codes before metatstatic diagnosis and up to 30 days past diagnosis will be mapped to the Elixhauser comorbidity index. ("Coding Algorithms for Defining Comorbidities in ICD-9-CM and ICD-10 Administrative Data"  by Quan et al is used as a guide for linking ICD codes to Elixhauser comorbidities.) Presence of concurrent or prior cancer diagnosis that is not lung cancer or metastasis and sites of metastases at time of diagnosis will also be created.** 

#### Elixhauser

In [289]:
diagnosis = pd.merge(diagnosis, enhanced_met[['PatientID', 'met_date']], on = 'PatientID', how = 'left')

In [290]:
diagnosis.loc[:, 'DiagnosisDate'] = pd.to_datetime(diagnosis['DiagnosisDate'])

In [291]:
diagnosis.loc[:, 'diagnosis_date_diff'] = (diagnosis['DiagnosisDate'] - diagnosis['met_date']).dt.days

In [292]:
# Remove decimal to make mapping to Elixhauser easier. 
diagnosis.loc[:, 'diagnosis_code'] = diagnosis['DiagnosisCode'].replace('\.', '', regex = True)

##### Elixhauser for ICD-9

In [293]:
# ICD-9 dataframe with unique codes for each patient. 
diagnosis_elix_9 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [294]:
row_ID(diagnosis_elix_9)

(13360, 2470)

In [295]:
diagnosis_elix_9.loc[:, 'chf'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('39891|'
                                                          '402(01|11|91)|'
                                                          '404(01|03|[19][13])|'
                                                          '42(5[456789]|8)'), 1, 0)
)

In [296]:
diagnosis_elix_9.loc[:, 'cardiac_arrhythmias'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('426([079]|1[023])|'
                                                          '427[012346789]|'
                                                          '7850|'
                                                          '996(01|04)|'
                                                          'V450|'
                                                          'V533'), 1, 0)
)

In [297]:
diagnosis_elix_9.loc[:, 'valvular_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('0932|'
                                                          '39[4567]|'
                                                          '424|'
                                                          '746[3456]|'
                                                          'V422|'
                                                          'V433'), 1, 0)
)

In [298]:
diagnosis_elix_9.loc[:, 'pulmonary_circulation'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('41(5[01]|6|7[089])'), 1, 0)
)

In [299]:
diagnosis_elix_9.loc[:, 'peripheral_vascular'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('0930|'
                                                          '4373|'
                                                          '44([01]|3[123456789]|71)|'
                                                          '557[19]|'
                                                          'V434'), 1, 0)
)

In [300]:
diagnosis_elix_9.loc[:, 'htn_uncomplicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('401'), 1, 0)
)

In [301]:
diagnosis_elix_9.loc[:, 'htn_complicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('40[2345]'), 1, 0)
)

In [302]:
diagnosis_elix_9.loc[:, 'paralysis'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('3341|'
                                                          '34([23]|4[01234569])'), 1, 0)
)

In [303]:
diagnosis_elix_9.loc[:, 'other_neuro_disorders'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('33(19|2[01]|3([45]|92)|[45]|62)|'
                                                          '34([015]|8[13])|'
                                                          '78[04]3'), 1, 0)
)

In [304]:
diagnosis_elix_9.loc[:, 'chronic_pulmonary'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('416[89]|'
                                                          '49|'
                                                          '50([012345]|64|8[18])'), 1, 0)
)

In [305]:
diagnosis_elix_9.loc[:, 'diabetes_uncomplicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('250[0123]'), 1, 0)
)

In [306]:
diagnosis_elix_9.loc[:, 'diabetes_complicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('250[456789]'), 1, 0)
)

In [307]:
diagnosis_elix_9.loc[:, 'hypothyroidism'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2409|'
                                                          '24([34]|6[18])'), 1, 0)
)

In [308]:
diagnosis_elix_9.loc[:, 'renal_failure'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('403[019]1|'
                                                          '404[019][23]|'
                                                          '58([56]|80)|'
                                                          'V4(20|51)|'
                                                          'V56'), 1, 0)
)

In [309]:
diagnosis_elix_9.loc[:, 'liver_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('070(2[23]]|3[23]|44|54|6|9)|'
                                                          '456[012]|'
                                                          '57([01]|2[2345678]|3[3489])|'
                                                          'V427'), 1, 0)
)

In [310]:
diagnosis_elix_9.loc[:, 'peptic_ulcer_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('53[1234][79]'), 1, 0)
)

In [311]:
diagnosis_elix_9.loc[:, 'aids_hiv'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('04[234]'), 1, 0)
)

In [312]:
diagnosis_elix_9.loc[:, 'lymphoma'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('20([012]|30)|'
                                                          '2386'), 1, 0)
)

In [313]:
diagnosis_elix_9.loc[:, 'metastatic_cancer'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('19[6789]'), 1, 0)
)

In [314]:
diagnosis_elix_9.loc[:, 'solid_tumor_wout_mets'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('1[456]|'
                                                          '17[012456789]|'
                                                          '18|'
                                                          '19([012345])'), 1, 0)
)

In [315]:
diagnosis_elix_9.loc[:, 'rheumatoid_arthritis'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('446|'
                                                          '7010|'
                                                          '71(0[0123489]|12|4|93)|'
                                                          '72([05]|85|889|930)'), 1, 0)
)

In [316]:
diagnosis_elix_9.loc[:, 'coagulopathy'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('28(6|7[1345])'), 1, 0)
)

In [317]:
diagnosis_elix_9.loc[:, 'obesity'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2780'), 1, 0)
)

In [318]:
diagnosis_elix_9.loc[:, 'weight_loss'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('26[0123]|'
                                                          '7832|'
                                                          '7994'), 1, 0)
)

In [319]:
diagnosis_elix_9.loc[:, 'fluid_electrolyte'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2(536|76)'), 1, 0)
)

In [320]:
diagnosis_elix_9.loc[:, 'blood_loss_anemia'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2800'), 1, 0)
)

In [321]:
diagnosis_elix_9.loc[:, 'deficiency_anemia'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('28(0[123456789]|1)'), 1, 0)
)

In [322]:
diagnosis_elix_9.loc[:, 'alcohol_abuse'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2652|'
                                                          '291[12356789]|'
                                                          '30(3[09]|50)|'
                                                          '3575|'
                                                          '4255|'
                                                          '5353|'
                                                          '571[0123]|'
                                                          '980|'
                                                          'V113'), 1, 0)
)

In [323]:
diagnosis_elix_9.loc[:, 'drug_abuse'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('292|'
                                                          '30(4|5[23456789])|'
                                                          'V6542'), 1, 0)
)

In [324]:
diagnosis_elix_9.loc[:, 'psychoses'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2938|'
                                                          '296[0145]4|'
                                                          '29[578]'), 1, 0)
)

In [325]:
diagnosis_elix_9.loc[:, 'depression'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('296[235]|'
                                                          '3(004|09|11)'), 1, 0)
)

In [326]:
# Create variable that captures ICD-9 codes not included in Elixhauser. 
diagnosis_elix_9.loc[:, 'elixhauser_other'] = (
    np.where(diagnosis_elix_9.iloc[:, 3:].eq(0).all(1), 1, 0)
)

In [327]:
# Percentage of ICD-9 codes not captured by Elixhauser.
diagnosis_elix_9['elixhauser_other'].sum()/len(diagnosis_elix_9)

0.6074101796407185

In [328]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities. 
diagnosis_elix_9_wide = (
    diagnosis_elix_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [329]:
row_ID(diagnosis_elix_9_wide)

(2470, 2470)

##### Elixhauser for ICD-10

In [330]:
# ICD-10 dataframe with unique codes for each patient.  
diagnosis_elix_10 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [331]:
row_ID(diagnosis_elix_10)

(31357, 4351)

In [332]:
diagnosis_elix_10.loc[:, 'chf'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I099|'
                                                           'I1(10|3[02])|'
                                                           'I255|'
                                                           'I4(2[056789]|3)|'
                                                           'I50|'
                                                           'P290'), 1, 0)
)

In [333]:
diagnosis_elix_10.loc[:, 'cardiac_arrhythmias'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I4(4[123]|5[69]|[789])|'
                                                           'R00[018]|'
                                                           'T821|'
                                                           'Z[49]50'), 1, 0)
)

In [334]:
diagnosis_elix_10.loc[:, 'valvular_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('A520|'
                                                           'I0([5678]|9[18])|'
                                                           'I3[456789]|'
                                                           'Q23[0123]|'
                                                           'Z95[234]'), 1, 0)
)

In [335]:
diagnosis_elix_10.loc[:, 'pulmonary_circulation'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I2([67]|8[089])'), 1, 0)
)

In [336]:
diagnosis_elix_10.loc[:, 'peripheral_vascular'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I7([01]|3[189]|71|9[02])|'
                                                           'K55[189]|'
                                                           'Z95[89]'), 1, 0)
)

In [337]:
diagnosis_elix_10.loc[:, 'htn_uncomplicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I10'), 1, 0)
)

In [338]:
diagnosis_elix_10.loc[:, 'htn_complicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I1[1235]'), 1, 0)
)

In [339]:
diagnosis_elix_10.loc[:, 'paralysis'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('G041|'
                                                           'G114|'
                                                           'G8(0[12]|[12]|3[012349])'), 1, 0)
)

In [340]:
diagnosis_elix_10.loc[:, 'other_neuro_disorders'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('G1[0123]|'
                                                           'G2([012]|5[45])|'
                                                           'G3(1[289]|[2567])|'
                                                           'G4[01]|'
                                                           'G93[14]|'
                                                           'R470|'
                                                           'R56'), 1, 0)
)

In [341]:
diagnosis_elix_10.loc[:, 'chronic_pulmonary'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I27[89]|'
                                                           'J4[01234567]|'
                                                           'J6([01234567]|84)|'
                                                           'J70[13]'), 1, 0)
)

In [342]:
diagnosis_elix_10.loc[:, 'diabetes_uncomplicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E1[01234][019]'), 1, 0)
)

In [343]:
diagnosis_elix_10.loc[:, 'diabetes_complicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E1[01234][2345678]'), 1, 0)
)

In [344]:
diagnosis_elix_10.loc[:, 'hypothyroidism'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E0[0123]|'
                                                           'E890'), 1, 0)
)

In [345]:
diagnosis_elix_10.loc[:, 'renal_failure'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I1(20|31)|'
                                                           'N1[89]|'
                                                           'N250|'
                                                           'Z49[012]|'
                                                           'Z9(40|92)'), 1, 0)
)

In [346]:
diagnosis_elix_10.loc[:, 'liver_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('B18|'
                                                           'I8(5|64)|'
                                                           'I982|'
                                                           'K7(0|1[13457]|[234]|6[023456789])|'
                                                           'Z944'), 1, 0)
)

In [347]:
diagnosis_elix_10.loc[:, 'peptic_ulcer_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('K2[5678][79]'), 1, 0)
)

In [348]:
diagnosis_elix_10.loc[:, 'aids_hiv'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('B2[0124]'), 1, 0)
)

In [349]:
diagnosis_elix_10.loc[:, 'lymphoma'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C8[123458]|'
                                                           'C9(0[02]|6)'), 1, 0)
)

In [350]:
diagnosis_elix_10.loc[:, 'metastatic_cancer'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C(7[789]|80)'), 1, 0)
)

In [351]:
diagnosis_elix_10.loc[:, 'solid_tumor_wout_mets'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C[01]|'
                                                           'C2[0123456]|'
                                                           'C3[01234789]|'
                                                           'C4[01356789]|'
                                                           'C5[012345678]|'
                                                           'C6|'
                                                           'C7[0123456]|'
                                                           'C97'), 1, 0)
)

In [352]:
diagnosis_elix_10.loc[:, 'rheumatoid_arthritis'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('L94[013]|'
                                                           'M0[568]|'
                                                           'M12[03]|'
                                                           'M3(0|1[0123]|[2345])|'
                                                           'M4(5|6[189])'), 1, 0)
)

In [353]:
diagnosis_elix_10.loc[:, 'coagulopathy'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D6([5678]|9[13456])'), 1, 0)
)

In [354]:
diagnosis_elix_10.loc[:, 'obesity'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E66'), 1, 0)
)

In [355]:
diagnosis_elix_10.loc[:, 'weight_loss'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E4[0123456]|'
                                                           'R6(34|4)'), 1, 0)
)

In [356]:
diagnosis_elix_10.loc[:, 'fluid_electrolyte'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E222|'
                                                           'E8[67]'), 1, 0)
)

In [357]:
diagnosis_elix_10.loc[:, 'blood_loss_anemia'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D500'), 1, 0)
)

In [358]:
diagnosis_elix_10.loc[:, 'deficiency_anemia'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D5(0[89]|[123])'), 1, 0)
)

In [359]:
diagnosis_elix_10.loc[:, 'alcohol_abuse'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F10|'
                                                           'E52|'
                                                           'G621|'
                                                           'I426|'
                                                           'K292|'
                                                           'K70[039]|'
                                                           'T51|'
                                                           'Z502|'
                                                           'Z7(14|21)'), 1, 0)
)

In [360]:
diagnosis_elix_10.loc[:, 'drug_abuse'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F1[12345689]|'
                                                           'Z7(15|22)'), 1, 0)
)

In [361]:
diagnosis_elix_10.loc[:, 'psychoses'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F2[0234589]|'
                                                           'F3([01]2|15)'), 1, 0)
)

In [362]:
diagnosis_elix_10.loc[:, 'depression'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F204|'
                                                           'F3(1[345]|[23]|41)|'
                                                           'F4[13]2'), 1, 0)
)

In [363]:
# Create variable that captures ICD-10 codes not included in Elixhauser. 
diagnosis_elix_10.loc[:, 'elixhauser_other'] = (
    np.where(diagnosis_elix_10.iloc[:, 3:].eq(0).all(1), 1, 0)
)

In [364]:
# Percenage of ICD-10 codes not captured by Elixhauser
diagnosis_elix_10['elixhauser_other'].sum()/len(diagnosis_elix_10)

0.5912874318334024

In [365]:
diagnosis_elix_10_wide = (
    diagnosis_elix_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [366]:
row_ID(diagnosis_elix_10_wide)

(4351, 4351)

In [367]:
# Merge Elixhauser 9 and 10 and sum by PatientID.
diagnosis_elixhauser = (
    pd.concat([diagnosis_elix_9_wide, diagnosis_elix_10_wide])
    .groupby('PatientID').sum()
)

In [368]:
# Create unqiue ICD count for each patient. 
diagnosis_elixhauser['icd_count'] = diagnosis_elixhauser.sum(axis = 1)  

In [369]:
# Other than unique ICD count, values greater than 1 are set to 1; 0 remains unchanged. 
diagnosis_elixhauser.iloc[:, :-1] = (
    diagnosis_elixhauser.iloc[:, :-1].mask(diagnosis_elixhauser.iloc[:, :-1] >1, 1)
)

In [370]:
diagnosis_elixhauser = diagnosis_elixhauser.reset_index()

In [371]:
row_ID(diagnosis_elixhauser)

(5922, 5922)

In [372]:
# Append missing training IDs.
diagnosis_elixhauser = (
    diagnosis_elixhauser
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(diagnosis_elixhauser['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [373]:
row_ID(diagnosis_elixhauser)

(6863, 6863)

#### Colon cancer location

**Right sided colon cancer (ie., proximal to the splenic flexure) is an independent, poor prognostic factor for overall survival. Using ICD codes, we will characterize the colon cancer as left vs. right sided.**

In [374]:
diagnosis_elix_9.loc[:, 'right_colon'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('153[01456]'), 1, 0)
)

In [375]:
diagnosis_elix_9.loc[:, 'left_colon'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('153[237]'), 1, 0)
)

In [376]:
colon_location_9 = diagnosis_elix_9[['PatientID', 'right_colon', 'left_colon']].groupby('PatientID').sum()

In [377]:
diagnosis_elix_10.loc[:, 'right_colon'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C18[01234]'), 1, 0)
)

In [378]:
diagnosis_elix_10.loc[:, 'left_colon'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C18[567]'), 1, 0)
)

In [379]:
colon_location_10 = diagnosis_elix_10[['PatientID', 'right_colon', 'left_colon']].groupby('PatientID').sum()

In [380]:
colon_location = (
    pd.concat([colon_location_9, colon_location_10])
    .groupby('PatientID').sum()
    .reset_index()
)

In [381]:
colon_location = pd.merge(enhanced_met[['PatientID', 'CrcSite']], colon_location, on = 'PatientID', how = 'left')

In [382]:
colon_location[['right_colon', 'left_colon']] = colon_location[['right_colon', 'left_colon']].fillna(0)

In [383]:
row_ID(colon_location)

(6863, 6863)

In [384]:
# Create indicator variable where where 2 if positive, 1 if negative, and 0 if unknown or missing. 
conditions = [
    (colon_location['CrcSite'] == 'Colon') & (colon_location['right_colon'] >= 1),
    (colon_location['CrcSite'] == 'Colon') & (colon_location['right_colon'] == 0) & (colon_location['left_colon'] >= 1),
    (colon_location['CrcSite'] == 'Colon') & (colon_location['right_colon'] == 0) & (colon_location['left_colon'] == 0),
    (colon_location['CrcSite'] == 'Rectum'),
    (colon_location['CrcSite'] == 'Colorectal NOS')]

choices = ['colon_right', 'colon_left', 'colon_unknown', 'rectum', 'unknown']

colon_location.loc[:, 'crc_site'] = np.select(conditions, choices)

In [385]:
enhanced_met = pd.merge(enhanced_met, colon_location[['PatientID', 'crc_site']], on = 'PatientID')

#### Other cancer 

##### ICD-9 Cancer codes 

In [386]:
# Select all ICD-9 cancer codes between 140-209.
# Exclude benign neoplasms: 210-229, carcinoma in site: 230-234, and neoplasms of uncertain behavior or nature: 235-239.
cancer_9 = (
    diagnosis_elix_9[diagnosis_elix_9['DiagnosisCode'].str.startswith(
        ('14','15', '16', '17', '18', '19', '20'))]
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [387]:
row_ID(cancer_9)

(3267, 2110)

**Remove the following ICD-9 codes representing colorectal cancer, metastasis, ill-defined neoplasms, and benign neoplasms of skin (BCC and SCC):**
* **153 - Malignant neoplasm of colon**
* **154 - Malignant neoplasm of rectum rectosigmoid junction and anus**
* **155 - Malignant neoplasm of liver and intrahepatic bile ducts**
* **158 - Malignant neoplasm of retroperitoneum and peritoneum**
* **159 - Malignant neoplasm of other and ill-defined sites within the digestive organs and peritoneum**
* **173 - Other and unspecified malignant neoplasm of skin**
* **195.2 - Malignant neoplasm of abdomen**
* **196 - Secondary and unspecified malignant neoplasm of lymph nodes**
* **197 - Secondary malignant neoplasm of respiratory and digestive systems**
* **198 - Secondary malignant neoplasm of other specified sites** 
* **199 - Malignant neoplasm without specification of site**

In [388]:
# Dataframe of ICD-9 neoplasm codes that exclude colorectal cancer, metastasis, or benign neoplasms.
other_cancer_9 = (
    cancer_9[~cancer_9['diagnosis_code'].str.match('15([34589])|'
                                                   '173|'
                                                   '19(52|[6789])')]
)

In [389]:
other_cancer_9.loc[:,'other_cancer_9'] = 1

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [390]:
other_cancer_9 = (
    other_cancer_9
    .drop_duplicates(subset = 'PatientID', keep = 'first')
    .filter(items = ['PatientID', 'other_cancer_9'])
)

In [391]:
row_ID(other_cancer_9)

(271, 271)

In [392]:
other_cancer_9 = (
    other_cancer_9
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(other_cancer_9['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [393]:
row_ID(other_cancer_9)

(6863, 6863)

##### ICD-10 Cancer codes

In [394]:
# Select all ICD-10 codes between C00-D49 
# Exclude in situ neoplasms: D00-D09, benign neoplasms: D10-D36, benign neuroendocrine tumor: D3A, and neoplasms of unspecified behavior: D37 and D49
cancer_10 = (
    diagnosis_elix_10[diagnosis_elix_10['DiagnosisCode'].str.startswith(
        ('C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D38', 'D39', 'D4'))]
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [395]:
row_ID(cancer_10)

(7494, 4073)

**Remove the following ICD-10 codes which capture lung cancer, metastasis, and benign skin neoplasms(eg., BCC and SCC).**
* **C18 - Malignant neoplasm of colon**
* **C19 - Malignant neoplasm of rectosigmoid junction**
* **C20 - Malignant neoplasm of rectum**
* **C21.8 - Malignant neoplasm of overlapping sites of rectum, anus and anal canal**
* **C22 - Malignant neoplasm of liver, not specified as primary or secondary**
* **C26 - Malignant neoplasm of other and ill-defined digestive organs**
* **C44 - Other and unspecified malignant neoplasm of skin**
* **C77 - Secondary and unspecified malignant neoplasm of lymph nodes**
* **C78 - Secondary malignant neoplasm of respiratory and digestive organs**
* **C79 - Secondary malignant neoplasm of other and unspecified sites**
* **C80 - Malignant neoplasm without specification of site**
* **D47.2 - Monoclonal gammopathy**
* **D48 - Neoplasm of uncertain behavior of other and unspecified sites**
* **D49 - Neoplasms of unspecified behavior** 

In [396]:
# Dataframe of ICD-10 neoplasm codes that exclude lung cancer, metastasis, or benign neoplasms.
other_cancer_10 = (
    cancer_10[~cancer_10['diagnosis_code'].str.match('C1[89]|'
                                                     'C2([06]|18|29)|'
                                                     'C44|'
                                                     'C7[789]|'
                                                     'C80|'
                                                     'D4(72|[89])')]
)

In [397]:
other_cancer_10.loc[:,'other_cancer_10'] = 1

In [398]:
# Drop duplicates.
other_cancer_10 = (
    other_cancer_10
    .drop_duplicates(subset = 'PatientID', keep = 'first')
    .filter(items = ['PatientID', 'other_cancer_10'])
)

In [399]:
row_ID(other_cancer_10)

(379, 379)

In [400]:
# Append missing training IDs.
other_cancer_10 = (
    other_cancer_10
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(other_cancer_10['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [401]:
row_ID(other_cancer_10)

(6863, 6863)

In [402]:
other_cancer = pd.merge(other_cancer_9, other_cancer_10, on = 'PatientID')

In [403]:
# Combine other_cancer_9 and other_cancer_19; replace values equal to 2 with 1. 
other_cancer = (
    other_cancer
    .assign(other_cancer = other_cancer['other_cancer_9'] + other_cancer['other_cancer_10'])
    .filter(items = ['PatientID', 'other_cancer'])
    .replace(2, 1)
)

In [404]:
row_ID(other_cancer)

(6863, 6863)

In [405]:
# Percentage of patients with a cancer other than colorectal or mets. 
len(other_cancer[other_cancer['other_cancer'] == 1])/len(other_cancer)

0.08873670406527757

#### Sites of metastases

##### ICD-9 sites of metastases

In [406]:
# Create dataframe contianing patients with ICD-9 codes within -90 to +30 days from advanced diagnosis and remove duplicate codes.(
diagnosis_mets_9 = (
    diagnosis
    .query('diagnosis_date_diff >= -90 and diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = ['PatientID', 'DiagnosisCode'], keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

**Sites of metastasis will be grouped into the following categories according to ICD-9 codes:**
* **Thorax - 197.0, 197.1, 197.2, and 197.3**
* **Peritoneum - 197.6**
* **Liver - 197.7**
* **Other GI - 197.4 and 197.8**
* **CNS - 198.3 and 198.4**
* **Bone - 198.5**
* **Other - 198.0, 198.1, 198.2, 198.6, 198.7, 198.8, and 196**

In [407]:
diagnosis_mets_9['thorax_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('197[0123]'), 1, 0)

In [408]:
diagnosis_mets_9['peritoneum_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('1976'), 1, 0)

In [409]:
diagnosis_mets_9['liver_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('1977'), 1, 0)

In [410]:
diagnosis_mets_9['other_gi_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('197[48]'), 1, 0)

In [411]:
diagnosis_mets_9['cns_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('198[34]'), 1, 0)

In [412]:
diagnosis_mets_9['bone_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('1985'), 1, 0)

In [413]:
diagnosis_mets_9['other_met'] = np.where(diagnosis_mets_9['diagnosis_code'].str.match('198[012678]|'
                                                                                      '196'), 1, 0)

In [414]:
# Collapse columns and sum.
diagnosis_mets_9 = (
    diagnosis_mets_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

##### ICD-10 sites of metastases

In [415]:
# Create dataframe contianing patients with ICD-10 codes within -90 to +30 days from advanced diagnosis and remove duplicate codes.
diagnosis_mets_10 = (
    diagnosis
    .query('diagnosis_date_diff >= -90 and diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = ['PatientID', 'DiagnosisCode'], keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

**Sites of metastasis will be grouped into the following categories according to ICD-10 codes:**
* **Thorax - C78.0, C78.1, C78.2, and C78.3**
* **Peritoneum - C78.6**
* **Liver - C78.7**
* **Other GI - C78.4 and C78.8**
* **CNS - C79.3 and C79.4**
* **Bone - C79.5**
* **Other - C77, C79.0, C79.1, C79.2, C79.6, C79.7, C79.8, and C79.9**

In [416]:
diagnosis_mets_10['thorax_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C78[0123]'), 1, 0)

In [417]:
diagnosis_mets_10['peritoneum_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C786'), 1, 0)

In [418]:
diagnosis_mets_10['liver_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C787'), 1, 0)

In [419]:
diagnosis_mets_10['other_gi_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C78[48]'), 1, 0)

In [420]:
diagnosis_mets_10['cns_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C79[34]'), 1, 0)

In [421]:
diagnosis_mets_10['bone_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C795'), 1, 0)

In [422]:
diagnosis_mets_10['other_met'] = np.where(diagnosis_mets_10['diagnosis_code'].str.match('C77|'
                                                                                        'C79[0126789]'), 1, 0)

In [423]:
# Collapse columns and sum. 
diagnosis_mets_10 = (
    diagnosis_mets_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [424]:
# Merge ICD-9 and ICD-10 mets tables; collapse and sum. 
diagnosis_mets = (
    pd.concat([diagnosis_mets_9, diagnosis_mets_10])
    .groupby('PatientID').sum()
)

In [425]:
# All values >1 replaced by 1. 
diagnosis_mets = (
    diagnosis_mets.mask(diagnosis_mets > 1, 1)
    .reset_index()
)

In [426]:
# Append missing training IDs.
diagnosis_mets = (
    diagnosis_mets.append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(diagnosis_mets['PatientID'])].to_frame(name = 'PatientID'), 
        sort = False)
    .fillna(0)
)

In [427]:
row_ID(diagnosis_mets)

(6863, 6863)

#### Merge

In [428]:
diagnosis_wide = pd.merge(diagnosis_elixhauser, other_cancer, on = 'PatientID')

In [429]:
diagnosis_wide = pd.merge(diagnosis_wide, diagnosis_mets, on = 'PatientID')

In [430]:
row_ID(diagnosis_wide)

(6863, 6863)

In [431]:
list(diagnosis_wide.columns)

['PatientID',
 'chf',
 'cardiac_arrhythmias',
 'valvular_disease',
 'pulmonary_circulation',
 'peripheral_vascular',
 'htn_uncomplicated',
 'htn_complicated',
 'paralysis',
 'other_neuro_disorders',
 'chronic_pulmonary',
 'diabetes_uncomplicated',
 'diabetes_complicated',
 'hypothyroidism',
 'renal_failure',
 'liver_disease',
 'peptic_ulcer_disease',
 'aids_hiv',
 'lymphoma',
 'metastatic_cancer',
 'solid_tumor_wout_mets',
 'rheumatoid_arthritis',
 'coagulopathy',
 'obesity',
 'weight_loss',
 'fluid_electrolyte',
 'blood_loss_anemia',
 'deficiency_anemia',
 'alcohol_abuse',
 'drug_abuse',
 'psychoses',
 'depression',
 'elixhauser_other',
 'icd_count',
 'other_cancer',
 'thorax_met',
 'peritoneum_met',
 'liver_met',
 'other_gi_met',
 'cns_met',
 'bone_met',
 'other_met']

In [432]:
%whos DataFrame

Variable                 Type         Data/Info
-----------------------------------------------
biomarker_wide           DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
cancer_10                DataFrame                PatientID Dia<...>\n[7494 rows x 3 columns]
cancer_9                 DataFrame                PatientID Dia<...>\n[3267 rows x 3 columns]
colon_location           DataFrame              PatientID CrcSi<...>\n[6863 rows x 5 columns]
colon_location_10        DataFrame                   right_colo<...>\n[4351 rows x 2 columns]
colon_location_9         DataFrame                   right_colo<...>\n[2470 rows x 2 columns]
demographics             DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
diagnosis                DataFrame                PatientID    <...>[218887 rows x 9 columns]
diagnosis_elix_10        DataFrame                PatientID Dia<...>[31357 rows x 37 columns]
diagnosis_elix_10_wide   DataFrame              PatientID 

In [433]:
# Keep biomarker_wide, demographics, diagnosis_wide, ecog_diagnosis_wide, enhanced_met, insurance_wide, 
# lab_wide, med_admin_wide, mortality, and weight_wide
del cancer_10
del cancer_9
del colon_location
del colon_location_10
del colon_location_9
del diagnosis
del diagnosis_elix_10
del diagnosis_elix_10_wide
del diagnosis_elix_9
del diagnosis_elix_9_wide
del diagnosis_elixhauser
del diagnosis_mets
del diagnosis_mets_10
del diagnosis_mets_9
del other_cancer
del other_cancer_10
del other_cancer_9

### SocialDeterminantsOfHealth

In [434]:
sdoh = pd.read_csv('SocialDeterminantsOfHealth.csv')

In [435]:
sdoh = sdoh[sdoh['PatientID'].isin(test_IDs)]

In [436]:
row_ID(sdoh)

(5903, 5903)

**Measures the area-level socioeconomic status (SES) of a patient between 2015 and 2019 based on their most recent address.**

In [437]:
conditions = [
    (sdoh['SESIndex2015_2019'] == '5 - Highest SES'),
    (sdoh['SESIndex2015_2019'] == '1 - Lowest SES')]    

choices = ['5', '1']
    
sdoh.loc[:, 'ses'] = np.select(conditions, choices, default = sdoh['SESIndex2015_2019'])

In [438]:
sdoh = sdoh.drop(columns = ['PracticeID', 'SESIndex2015_2019'])

In [439]:
sdoh_wide = (
    sdoh
    .append(
        pd.Series(test_IDs)[~pd.Series(test_IDs).isin(sdoh['PatientID'])].to_frame(name = 'PatientID'),
        sort = False)
)

In [440]:
row_ID(sdoh_wide)

(6863, 6863)

In [441]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6863 rows x 5 columns]
demographics          DataFrame              PatientID gende<...>\n[6863 rows x 7 columns]
diagnosis_wide        DataFrame              PatientID  chf <...>n[6863 rows x 42 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6863 rows x 2 columns]
enhanced_met          DataFrame              PatientID diagn<...>\n[6863 rows x 8 columns]
insurance_wide        DataFrame              PatientID  medi<...>\n[6863 rows x 9 columns]
lab_wide              DataFrame              PatientID  albu<...>[6863 rows x 137 columns]
med_admin_wide        DataFrame              PatientID  ster<...>n[6863 rows x 15 columns]
mortality             DataFrame              PatientID  deat<...>\n[6863 rows x 3 columns]
sdoh                  DataFrame               PatientID ses\<...>\n[5903 rows x 2 columns]


In [442]:
# Keep biomarker_wide, demographics, diagnosis_wide, ecog_diagnosis_wide, enhanced_met, insurance_wide, 
# lab_wide, med_admin_wide, mortality, sdoh_wide, and weight_wide
del sdoh

## Part 3: File merge

In [443]:
enhanced_met = enhanced_met.drop(columns = ['diagnosis_date', 'met_date', 'CrcSite'])

In [444]:
test_full = pd.merge(demographics, enhanced_met, on = 'PatientID')

In [445]:
test_full = pd.merge(test_full, mortality, on = 'PatientID')

In [446]:
test_full = pd.merge(test_full, med_admin_wide, on = 'PatientID')

In [447]:
test_full = pd.merge(test_full, biomarker_wide, on = 'PatientID')

In [448]:
test_full = pd.merge(test_full, insurance_wide, on = 'PatientID')

In [449]:
test_full = pd.merge(test_full, ecog_diagnosis_wide, on = 'PatientID')

In [450]:
test_full = pd.merge(test_full, weight_wide, on = 'PatientID')

In [451]:
test_full = pd.merge(test_full, lab_wide, on = 'PatientID')

In [452]:
test_full = pd.merge(test_full, diagnosis_wide, on = 'PatientID')

In [453]:
test_full = pd.merge(test_full, sdoh_wide, on = 'PatientID')

In [454]:
row_ID(test_full)

(6863, 6863)

In [455]:
len(test_full.columns)

224

In [456]:
list(test_full.columns)

['PatientID',
 'gender',
 'race',
 'ethnicity',
 'age',
 'p_type',
 'region',
 'stage',
 'met_year',
 'delta_met_diagnosis',
 'crc_site',
 'death_status',
 'timerisk_activity',
 'steroid_diag',
 'opioid_PO_diag',
 'nonopioid_PO_diag',
 'pain_IV_diag',
 'ac_diag',
 'antiinfective_IV_diag',
 'antiinfective_diag',
 'antihyperglycemic_diag',
 'ppi_diag',
 'antidepressant_diag',
 'bta_diag',
 'thyroid_diag',
 'is_diag',
 'adjuv',
 'KRAS',
 'dMMR_MSIh',
 'NRAS',
 'BRAF',
 'medicare',
 'medicaid',
 'medicare_medicaid',
 'commercial',
 'patient_assistance',
 'other_govt',
 'self_pay',
 'other',
 'ecog_diagnosis',
 'weight_diag',
 'bmi_diag',
 'bmi_diag_na',
 'weight_pct_change',
 'weight_pct_na',
 'weight_slope',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'cea_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_di

In [457]:
test_full.to_csv('test_full.csv', index = False, header = True)